<a href="https://colab.research.google.com/github/SanyaKapoor/Site-Level-Impact-Assessment-of-Farm-Ponds/blob/main/DID_%7C_RQ_1_%7C_AEZ_12_(consolidated_new_pipeline).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the script that is used to collate all co-variates and the differences in differences results for any AEZ in India.

Pre-requisite knowledge:
✅- Look for this symbol anywhere in this script - this denotes a place to change values/ this is a place to customise the script as per your AEZ. FIRST CHECK ALL THE ✅s, MAKE ANY UPDATES/FULFILL ANY REQUIREMENTS OF CHECKBOXES.AFTER THAT - RUN ALL CELLS IN THE ORDER IN WHICH THEY APPEAR IN THIS SCRIPT. Note - In some places you may need to wait till the earth engine tasks complete in background/manually upload a file before you move ahead.

TODO - Add Expeceted running time for sample of 10k and section descriptions

#Step 0 - Imports and Initialisations

In [ ]:
!pip3 install geojson
!pip3 install geemap==0.20.7
!pip3 install rasterio
!pip install ipyleaflet==0.16
!pip3 install psmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.8 MB/s eta 0:00:00
  Attempting uninstall: geemap
    Found existing installation: geemap 0.34.5
    Uninstalling geemap-0.34.5:
      Successfully uninstalled geemap-0.34.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.4 MB/s eta 0:00:00
  Attempting uninstall: ipyleaflet
    Found existing installation: ipyleaflet 0.19.2
    Uninstalling ipyleaflet-0.19.2:
      Successfully uninstalled ipyleaflet-0.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geemap 0.20.7 requires ipyleaflet>=0.17.0, but you have ipyleaflet 0.16.0 which is incompatible.


In [ ]:
from google.colab import drive
import ee
import geemap
import geemap.colormaps as cm
import rasterio
from rasterio.plot import show
import os
import time
import sys

from scipy.stats import ttest_ind
import numpy as np
import pandas as pd
import json
import math

from datetime import date, datetime

import matplotlib.pyplot as plt
import seaborn as sns

from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *

import geopy.distance
import statsmodels.formula.api as smf

from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer

In [ ]:
ee.Authenticate(force=True)
ee.Initialize(project='ee-mtpictd')
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


✅ Add the folder where you want to save all your results 🔽

In [ ]:
%cd /content/drive/MyDrive/Sanya/DID-AEZ-12

/content/drive/MyDrive/Sanya/DID-AEZ-12


✅Change aoi-aer-regcode to the AEZ number

In [ ]:
def getDwLayer(work_year, aoi_boundary):
  interval = (str(int(work_year)-1)+'-'+cropping_year_start, str(int(work_year))+'-'+cropping_year_end)
  DynamicWorld_dataset = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(interval[0], interval[1]).filterBounds(aoi_boundary).mode().clip(aoi_boundary)
  dwComposite = DynamicWorld_dataset.select('label')

  return dwComposite

In [ ]:
aoi_aer_regcode = 12
work_type = 'Farm Pond'
work_year = '2017'
seasonality = None
no_sampling = 500
confounding_vars = ['elevation',
                    'slope',
                    'dist_closest_crop',
                    'dist_closest_river',
                    'flow_accumulation',
                    'dist_closest_lin',
                    'dist_closest_road',
                    'recent_cropping_intensity',
                    'dist_closest_upstream_forest',
                    'proximity_water']
bufer_size = 500
dem_resolution = 30
dw_resolution = 10
cropping_year_start = '07-01'
cropping_year_end = '06-30'
kharif = {"start": "07-01", "end": "10-31"}
rabi = {"start": "11-01", "end": "02-30"}
zaid = {"start": "03-01", "end": "06-30"}
cropping_seasons = ["kharif", "rabi", "zaid"]
ndwi_viz = {
        "min": 0,
        "max": 5000,
        "palette": [
            "#be254a", "#dc484c", "#ef6645", "#f88c51", "#fdb365", "#fed27f", "#feeb9d", "#fffebe",
            "#f0f9a7", "#d8ef9b", "#b3e0a2", "#89d0a4", "#60bba8", "#3f97b7", "#4273b3"
        ]
    }
lulc_viz = {
    "min": 0,
    "max": 8,
    "palette": [
        "#419BDF", "#397D49", "#88B053", "#7A87C6", "#E49635", "#DFC35A",
        "#C4281B", "#A59B8F", "#B39FE1"
        ]
    }
india_district_boundaries = ee.FeatureCollection('projects/ee-mtpictd/assets/india_district_boundaries')
india_block_boundaries_gee = "projects/ee-anz208490/assets"
agro_eco_regions_gee = "users/mtpictd/agro_eco_regions"
drainage_density_file = 'projects/ee-priyadarshini/assets/india_drainage_density'
lineament_file = "projects/ee-harshita-om/assets/india_lineaments"
flow_accumulation_file = "users/mtpictd/flow_acc_india"
india_roads_file = "users/mtpictd/india_roads_gfbk"
india_rivers_file = "users/mtpictd/india_rivers"
agro_eco_region = ee.FeatureCollection(agro_eco_regions_gee).filter(ee.Filter.eq('ae_regcode', aoi_aer_regcode))
agro_eco_region
aoi_boundary = agro_eco_region
aoi_boundary_geometry = agro_eco_region.geometry()
dwComposite = getDwLayer(work_year, aoi_boundary_geometry)

#Step 1 - LULC

On the earth engine console for the account ictdiitd@gmail.com a team member has already computed the LULC assets for all AEZs.

✅ - Seek help to login to the account. Identify the computed assets for the AEZ - For each computed asset (There would be around 20 for each AEZ) - Click on share and paste mtpictd@gmail.com in the prompt, this will allow you to access all the computed files for that AEZ through this account.

#Step 2 - Generate Df and compute covariates

##Methods for computing covariates

In [ ]:
def calc_slope(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  SRTM_dataset = ee.Image("USGS/SRTMGL1_003")
  elevation = SRTM_dataset.select('elevation').clip(aoi_boundary)
  slope = ee.Terrain.slope(elevation).clip(aoi_boundary)
  data_points_bfd = data_points.map(lambda f: f.buffer(100))
  slope_coords=slope.reduceRegions(data_points_bfd, ee.Reducer.mean().setOutputs(["slope"]), scale=dem_resolution)
  return slope_coords

In [ ]:
def calc_drainage_density(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  drainage_density = ee.Image(drainage_density_file).clip(aoi_boundary)
  dd_coords=drainage_density.reduceRegions(data_points, ee.Reducer.first().setOutputs(["drainage_density"]), scale=dem_resolution)
  return dd_coords

✅ Change LULC input along with the boundary. When you change AEZ, you need to use its LULC

In [ ]:
#Check if asset is accessible
#XYZ = ee.Image("projects/ee-ictdiitd/assets/AEZ_12_"+str(2018)+"-"+cropping_year_start+"_"+str(2019)+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
#print(XYZ)
text = ee.Image("projects/ee-indiasat/assets/LULC_Version2_Outputs_NewHierarchy/AEZ_"+str(12)+"_"+str(2016)+"-"+cropping_year_start+"_"+str(2017)+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
print(text)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "users/mtpictd/agro_eco_regions"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.equals",
                "arguments": {
                  "leftField": {
                    "constantValue": "ae_regcode"
                  },
                  "rightValue": {
                    "constantValue": 12
                  }
                }
              }
            }
          }
        }
      },
      "input": {
       

✅Change the AEZ number

In [ ]:
def calc_recent_cropping_intensity(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  cropping_intensity = ee.Image("projects/ee-ictdiitd/assets/AEZ_12_"+str(int(work_year)-1)+"-"+cropping_year_start+"_"+str(int(work_year))+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
  cropping_intensity = cropping_intensity.remap([8,9,10,11], [1,1,2,3], 0)
  cropping_intensity = cropping_intensity.selfMask()
  data_points_bfd = data_points.map(lambda f: f.buffer(100))
  cs_coords=cropping_intensity.reduceRegions(data_points_bfd, ee.Reducer.mode().setOutputs(["recent_cropping_intensity"]), scale=dem_resolution)
  return cs_coords

In [ ]:
def calc_dist_closest_crop(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  filtered_crop = dwComposite.eq(4).selfMask()  # Getting crop layer
  distance = filtered_crop.fastDistanceTransform().sqrt().multiply(ee.Image.pixelArea().sqrt()).rename("distance")  # Distance of each pixel to non-zero valued pixel in image
  coords_distToNearestCrop=distance.reduceRegions(data_points, ee.Reducer.first().setOutputs(["dist_closest_crop"]), scale=dw_resolution)
  return coords_distToNearestCrop

In [ ]:
def calc_flow_accumulation(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  flow_acc = ee.Image(flow_accumulation_file).clip(aoi_boundary)
  fa_coords=flow_acc.reduceRegions(data_points, ee.Reducer.first().setOutputs(["flow_accumulation"]), scale=dem_resolution)
  return fa_coords

In [ ]:
def calc_dist_closest_lin(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  lin_layer = ee.Image(lineament_file).clip(aoi_boundary)
  distance = lin_layer.fastDistanceTransform().sqrt().multiply(ee.Image.pixelArea().sqrt()).rename("distance")
  coords_distToNearestLin=distance.reduceRegions(data_points, ee.Reducer.first().setOutputs(["dist_closest_lin"]), scale=dem_resolution)
  return coords_distToNearestLin

In [ ]:
def calc_dist_closest_road(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  road_layer = ee.FeatureCollection(india_roads_file).filterBounds(aoi_boundary)
  distance = road_layer.distance().rename("distance")
  coords_distToNearestRoad=distance.reduceRegions(data_points, ee.Reducer.first().setOutputs(["dist_closest_road"]), scale=dem_resolution)
  return coords_distToNearestRoad

In [ ]:
def calc_dist_closest_river(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  river_layer = ee.FeatureCollection(india_rivers_file).filterBounds(aoi_boundary)
  distance = river_layer.distance().rename("distance")
  coords_distToNearestRiver=distance.reduceRegions(data_points, ee.Reducer.first().setOutputs(["dist_closest_river"]), scale=dem_resolution)
  return coords_distToNearestRiver

In [ ]:
def calc_dist_closest_upstream_forest(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  filtered_forest = dwComposite.eq(1).selfMask()
  SRTM_dataset = ee.Image("USGS/SRTMGL1_003")
  elevation = SRTM_dataset.select('elevation').clip(aoi_boundary)
  data_points_bfd = data_points.map(lambda f: f.buffer(100))
  elev_coords=elevation.reduceRegions(data_points_bfd, ee.Reducer.mean().setOutputs(["elevation"]), scale=dem_resolution)
  def setDistCForest(f):
    elev_f = ee.Number(f.get('elevation')).toFloat()
    filtered_elevation = elevation.updateMask(elevation.gt(elev_f))
    upstream_forest = (filtered_forest.mask(filtered_elevation)).selfMask()
    distance_upstreamforest = upstream_forest.fastDistanceTransform().sqrt().multiply(ee.Image.pixelArea().sqrt()).rename("distance")
    distCUForest = distance_upstreamforest.reduceRegions(ee.FeatureCollection(f), ee.Reducer.first().setOutputs(["dist_closest_upstream_forest"]), scale=dw_resolution)
    return distCUForest
  distCUForest_fc = elev_coords.map(lambda f: setDistCForest(f)) #10K points 10K feature c ka feature c
  flattened_distCUForest_fc = distCUForest_fc.flatten()
  return flattened_distCUForest_fc

In [ ]:
def calc_elevation(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  SRTM_dataset = ee.Image("USGS/SRTMGL1_003")
  elevation = SRTM_dataset.select('elevation').clip(aoi_boundary)
  data_points_bfd = data_points.map(lambda f: f.buffer(100))
  elev_coords=elevation.reduceRegions(data_points_bfd, ee.Reducer.mean().setOutputs(["elevation"]), scale=dem_resolution)
  return elev_coords

In [ ]:
def calc_proximity_water(data_points, aoi_boundary, dwComposite, waterworks_aoi):
  filtered_water = dwComposite.eq(0).selfMask()  # Getting water layer
  def sumListElements(cur, prev):
    prevVal = ee.Number(ee.List(prev).get(-1))
    curVal = prevVal.add(cur)
    return ee.List(prev).add(curVal)
  buffer_ranges = [100,200,400,800,1600]
  weights = []
  for b in buffer_ranges:
    weights.append(1/(b**2))
  sum_weights = [0]
  sum_weights = ee.List(ee.List(weights).iterate(sumListElements, sum_weights)).get(-1)
  def setProximityWater(f):
    weighted_water_pixels = [0]
    for i, buffer in enumerate(buffer_ranges):
      buffer_geom = ee.FeatureCollection(f.buffer(buffer)).geometry()
      if i>0:
        buffer_geom = buffer_geom.difference(ee.FeatureCollection(f.buffer(buffer_ranges[i-1])).geometry())
        water_mask_buffer = filtered_water.reduceToVectors(geometry=buffer_geom, crs=filtered_water.projection(), scale=dw_resolution)
        count_water_pixels = water_mask_buffer.aggregate_array('count')
        sum_water_pixels = [0]
        sum_water_pixels = ee.List(count_water_pixels.iterate(sumListElements, sum_water_pixels))
        water_pixels_buffer = sum_water_pixels.get(-1)
        wtd_water_pixels_buffer = ee.Number(water_pixels_buffer).multiply(weights[i])
        weighted_water_pixels = ee.List(weighted_water_pixels).add(wtd_water_pixels_buffer)
    proximity_water_f = [0]
    proximity_water_f = ee.Number(ee.List(weighted_water_pixels.iterate(sumListElements, proximity_water_f)).get(-1)).divide(sum_weights)
    return f.set({
        'proximity_water': proximity_water_f
    })
  proximityWater = data_points.map(lambda f: setProximityWater(f))
  return proximityWater

##Farm Ponds

In [ ]:
all_farmponds = ee.FeatureCollection("projects/ee-kshitiz/assets/farmponds_india_compact")

In [ ]:
aoi_farmponds = all_farmponds.filterBounds(aoi_boundary_geometry)

✅Change the AEZ number

In [ ]:
folder_run = 'DID-AEZ-12'
fn = "farmponds_aoi"
export_task = ee.batch.Export.table.toDrive(collection=aoi_farmponds, folder=folder_run, description = fn+folder_run,
  fileNamePrefix = fn, fileFormat='CSV')
export_task.start()

In [ ]:
aoi_farmponds_df = pd.read_csv("farmponds_aoi.csv")

In [ ]:
aoi_farmponds_df

,system:index,Asset ID,Asset Name,Block,District,Gram_panchayat,Panchayat,Panchayat_ID,State,Total_Expenditure,Work Code,Work Name,Work Type,Work_start_date,creation_time,.geo
0,0000000000000001b827,2006446648,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,ANDIBHA,ANDIBHA,129976,ANDHRA PRADESH,42107.0,0203003033/WC/135170017,Farm Pond,Construction of Farm Ponds for Individuals,17/04/2017,2021-02-16 07:35:26.609205,"{""type"":""Point"",""coordinates"":[82.728270473143..."
1,0000000000000001b828,2006446650,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,ANDIBHA,ANDIBHA,129976,ANDHRA PRADESH,40670.0,0203003033/WC/159170029,Farm Pond,Construction of Farm Ponds for Individuals,19/06/2017,2021-02-15 17:38:02.546993,"{""type"":""Point"",""coordinates"":[82.732698360489..."
2,0000000000000001b829,2006446651,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,ANDIBHA,ANDIBHA,129976,ANDHRA PRADESH,40293.0,0203003033/WC/159170030,Farm Pond,Construction of Farm Ponds for Individuals,17/04/2017,2021-02-15 17:43:06.688639,"{""type"":""Point"",""coordinates"":[82.732720655994..."
3,0000000000000001b82a,2005520196,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,Addumanda,Addumanda,129977,ANDHRA PRADESH,318782.0,0203003020/WC/092170132,Farm Pond,Construction of Farm Ponds for Individuals,15/05/2017,2018-04-13 19:34:46.629,"{""type"":""Point"",""coordinates"":[82.693676767170..."
4,0000000000000001b82b,2005520197,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,Addumanda,Addumanda,129977,ANDHRA PRADESH,228909.0,0203003020/WC/092170133,Farm Pond,Construction of Farm Ponds for Individuals,15/05/2017,2018-06-15 18:14:00.828,"{""type"":""Point"",""coordinates"":[82.698728928664..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25474,00010000000000013a4b,18001338011,1829002037/WC/473424855,bhadravati,CHANDRAPUR,MUDHOLI,mudholi,149496,MAHARASHTRA,373688.0,1829002037/WC/473424855,FP Shetatle Mudholi Ashok Sonwane,Farm Pond,10/01/2017,2019-07-19 16:18:19.46311,"{""type"":""Point"",""coordinates"":[79.172899889300..."
25475,00010000000000013a4c,18001338016,1829002037/WC/473424856,bhadravati,CHANDRAPUR,MUDHOLI,mudholi,149496,MAHARASHTRA,374062.0,1829002037/WC/473424856,FP Shetatle Mudholi Budhaji Gaikwad,Farm Pond,10/01/2017,2019-07-19 13:57:54.123102,"{""type"":""Point"",""coordinates"":[79.176199624079..."
25476,00010000000000013a4d,18001250900,1829002038/WC/473423704,bhadravati,CHANDRAPUR,VAYGAON (TU),vaygaon (tu),149516,MAHARASHTRA,121901.0,1829002038/WC/473423704,BN Bodi Nutanikarn Ratnapur ri Gulab Bhadke,Farm Pond,03/01/2017,2019-07-19 14:00:45.213886,"{""type"":""Point"",""coordinates"":[79.174246537818..."
25477,00010000000000013a4e,18001520571,JSV SINCHAN VIHIR BANDU SITARAM PARSUTKAR SAKH...,rajura,CHANDRAPUR,SAKHARWAHI(Rajura),sakharwahi(rajura),150071,MAHARASHTRA,266799.0,1829010016/IF/1234803133,JSV सिंचन विहीर बंडू सीताराम परसूटकर साखरवाही ...,Farm Pond,13/05/2017,10-FEB-20 17:04:41.032804,"{""type"":""Point"",""coordinates"":[79.318257665440..."


##Generating treated and control groups

###Treated group

In [ ]:
def setLatLonTreat(feature):
  coords = feature.geometry().coordinates()
  return feature.set({
      'lon':coords.get(0),
      'lat':coords.get(1),
      'Treatment':1,
      })

def getTreatedGroup(aoi_boundary, aoi_farmponds, seasonality = -1):
  assets = aoi_farmponds.map(setLatLonTreat)

  assets_ee = ee.FeatureCollection([])
  cs_coords = calc_recent_cropping_intensity(assets, aoi_boundary, dwComposite, None)

  if seasonality != -1: ## 1 or 2 or 3 ##
    ## If seasonality is specified, only get treated points where seasonality equals argument ##
    assets_with_cs = cs_coords.filter(ee.Filter.eq('recent_cropping_intensity', seasonality))
    # assets_df = geemap.ee_to_pandas(assets_with_cs)       # exceeds limit when AEZ bigger in size, so phasing out this requirement. Keep ee assets only

  else: ## 1 + 2 + 3 ##
    assets_with_cs = cs_coords.filter(ee.Filter.notNull(['recent_cropping_intensity']))
    # assets_df = geemap.ee_to_pandas(assets_with_cs)

  return assets_with_cs

✅Change the AEZ number

In [ ]:
folder_run = 'DID-AEZ-12'
seasonality = -1
assets_ee = getTreatedGroup(aoi_boundary, aoi_farmponds, seasonality) # assets to analyze
print(assets_ee.size().getInfo())

24610


In [ ]:
folder_run = 'DID-AEZ-12'
fn = "treated_df"
export_task = ee.batch.Export.table.toDrive(collection=assets_ee, folder=folder_run, description = fn+folder_run,
  fileNamePrefix = fn, fileFormat='CSV', selectors=['lat','lon','Asset ID','Asset Name','Block','District','Panchayat','Total_Expenditure','Treatment', 'State'])
export_task.start()

In [ ]:
folder_run = 'DID-AEZ-12'
treated = pd.read_csv("treated_df.csv")
treated = treated.set_index([pd.Index(range(len(treated)))])
treated['idx'] = treated.index
treated = treated.drop_duplicates(subset='Asset ID', keep="first")
treated

,lat,lon,Asset ID,Asset Name,Block,District,Panchayat,Total_Expenditure,Treatment,State,idx
0,18.099233,82.728270,2006446648,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,ANDIBHA,42107.0,1,ANDHRA PRADESH,0
1,18.099442,82.732698,2006446650,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,ANDIBHA,40670.0,1,ANDHRA PRADESH,1
2,18.099500,82.732721,2006446651,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,ANDIBHA,40293.0,1,ANDHRA PRADESH,2
3,18.112828,82.693677,2005520196,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,Addumanda,318782.0,1,ANDHRA PRADESH,3
4,18.139298,82.698729,2005520197,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,Addumanda,228909.0,1,ANDHRA PRADESH,4
...,...,...,...,...,...,...,...,...,...,...,...
24605,20.091425,79.172900,18001338011,1829002037/WC/473424855,bhadravati,CHANDRAPUR,mudholi,373688.0,1,MAHARASHTRA,24605
24606,20.087109,79.176200,18001338016,1829002037/WC/473424856,bhadravati,CHANDRAPUR,mudholi,374062.0,1,MAHARASHTRA,24606
24607,20.088857,79.174247,18001250900,1829002038/WC/473423704,bhadravati,CHANDRAPUR,vaygaon (tu),121901.0,1,MAHARASHTRA,24607
24608,19.709838,79.318258,18001520571,JSV SINCHAN VIHIR BANDU SITARAM PARSUTKAR SAKH...,rajura,CHANDRAPUR,sakharwahi(rajura),266799.0,1,MAHARASHTRA,24608


In [ ]:
treated_blocks = treated[['Block','District','State']]
treated_blocks = treated_blocks.value_counts(['Block','District','State']).reset_index(name='counts')
treated_blocks

,Block,District,State,counts
0,govindpur,DHANBAD,JHARKHAND,719
1,masaliya,DUMKA,JHARKHAND,527
2,dumka,DUMKA,JHARKHAND,482
3,gandey,GIRIDIH,JHARKHAND,440
4,giridih,GIRIDIH,JHARKHAND,388
...,...,...,...,...
319,maneswar,SAMBALPUR,ODISHA,1
320,biramaharajpur,SONEPUR,ODISHA,1
321,bijepur,BARGARH,ODISHA,1
322,bhanjanagar,GANJAM,ODISHA,1


###Control Group

In [ ]:
# Getting waterworks because we buffer out water bodies when sampling control points. Only want them from cropping areas
all_waterworks = ee.FeatureCollection("projects/ee-kshitiz/assets/waterworks_india_compact")
aoi_waterworks = all_waterworks.filterBounds(aoi_boundary_geometry)

In [ ]:
aoi_waterworks.size().getInfo()

40601

✅Change the AEZ number

In [ ]:
def addPropertiesControls(f):
  coords = f.geometry().coordinates()
  return f.set({
      'Treatment': 0,
      'lat': coords.get(-1),
      'lon': coords.get(0)
      })

def getAssetMask(aoi_boundary, waterworks_aoi = []):
  ## Buffering around every NREGA water asset built to mask for control points ##
  assets_all_ee = aoi_waterworks        # defined above in prev cell. ignore the function argument. Updated functionality.

  buffered = assets_all_ee.map(lambda f: f.buffer(500))

  aoi_bg = ee.Image(1).clip(aoi_boundary)
  buffers = ee.Image(1).clip(aoi_boundary).clipToCollection(buffered)
  masked_buffers = aoi_bg.updateMask(buffers.unmask(0).Not())

  return masked_buffers


# the block or district boundaries asset has these column names
def setPropertiesControl(block_fc):
  def iterateOn(f):
    return f.set({
        'Block':block_fc.get('Subdistric'),
        'District':block_fc.get('District N'),
        'State':block_fc.get('State Name')})
  return iterateOn


def getControlGroup(dwComposite, aoi_boundary, waterworks_aoi, treated_blocks, seasonality = -1):
  asset_masked_aoi = getAssetMask(aoi_boundary)
  composite_mask = asset_masked_aoi

  filtered = dwComposite.lt(6).And(dwComposite.gt(0)).Or(dwComposite.eq(7)).selfMask().clip(aoi_boundary) # Filtering out water, built up, snow_and_ice layers
  inverse_filtered = filtered.eq(0)

  # add 500m buffer to all masked areas (non 0) here
  inverse_filtered_buffered = inverse_filtered.focal_max(
      radius = 500,
      kernelType = 'circle',
      units = 'meters',
      iterations = 1
  )  # note: focal_max has a limit of 256 pixels. Since resolution for DW is 10m, we are doing for 50 pixels (which is okay)
    # max radius we can put is 2560 meters

  filtered_buffered = inverse_filtered_buffered.eq(0)
  composite_mask = filtered_buffered.updateMask(asset_masked_aoi)


    ### NOTE - change LULC file with new boundary/aez

# lulc_image = ee.Image("projects/ee-indiasat/assets/LULC_Final_Outputs_NewHierarchy/AEZ_13_"+str(2021)+"-"+cropping_year_start+"_"+str(2022)+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
  lulc_image = ee.Image("projects/ee-ictdiitd/assets/AEZ_12_"+str(int(work_year)-1)+"-"+cropping_year_start+"_"+str(int(work_year))+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
  #lulc_image = ee.Image("projects/ee-indiasat/assets/LULC_Version2_Outputs_NewHierarchy/AEZ_12_lulc_"+str(2021)+"-"+cropping_year_start+"_"+str(2022)+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
  filtered_lulc = lulc_image.gt(4).selfMask().clip(aoi_boundary) # Filtering out water & built up
  inverse_filtered_lulc = filtered_lulc.eq(0)
  # filtered_lulc = lulc_image.gt(4).Or(lulc_image.eq(0)).selfMask().clip(aoi_boundary) # Filtering out water

  inverse_filtered_lulc_buffered = inverse_filtered_lulc.focal_max(
      radius = 500,
      kernelType = 'circle',
      units = 'meters',
      iterations = 1
  )

  filtered_lulc_buffered = inverse_filtered_lulc_buffered.eq(0)
  composite_mask = composite_mask.updateMask(filtered_lulc_buffered)

  ci = ee.Image("projects/ee-ictdiitd/assets/AEZ_12_"+str(int(work_year)-1)+"-"+cropping_year_start+"_"+str(int(work_year))+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
  #ci = ee.Image("projects/ee-indiasat/assets/LULC_Version2_Outputs_NewHierarchy/AEZ_12_lulc_"+str(int(work_year)-1)+"-"+cropping_year_start+"_"+str(int(work_year))+"-"+cropping_year_end+"_LULCmap_10m").select('predicted_label').clip(aoi_boundary)
  # will need to get this data for other AEZs
  ci = ci.remap([8,9,10,11], [1,1,2,3], 0)     # remap the lulc classes as per cropping intensity. The mapping can be found in LULC script for each label
  ci = ci.selfMask()
  ci_reduced = ci.reduceNeighborhood(reducer=ee.Reducer.mode(), kernel=ee.Kernel.circle(radius=100, units='meters'))

  if seasonality != -1:
    ci_seasonality = ci_reduced.eq(seasonality).selfMask()
    composite_mask = composite_mask.updateMask(ci_seasonality)

  else: ## Get control points from only croplands ##
    composite_mask = composite_mask.updateMask(ci_reduced)


    ## Not all block boundaries available. The block boundaries that is available, does not have all blocks in the NREGA data (treated points)
    # So to counter this, we take the mean lat lon of all treated points in a block, and add a 5km buffer to it, which allows us to approximate the block boundary.
    # We then sample control points from this boundary.

  control_points_ee = ee.FeatureCollection([])
  for i in treated_blocks.index:
    state, block, district, count = treated_blocks['State'][i], treated_blocks['Block'][i], treated_blocks['District'][i], int(treated_blocks['counts'][i])

    treated_block = treated[treated['Block'] == block]
    mean_lat = treated_block['lat'].mean()
    mean_lon = treated_block['lon'].mean()

    center_point = ee.Geometry.Point([mean_lon, mean_lat])
    # add 5km buffer to the point
    center_point_buffered = center_point.buffer(5000)

    #create feature from geometry
    block_fc = ee.Feature(center_point_buffered, {'Subdistric': block, 'District N': district, 'State Name': state})

    block_pts = composite_mask.stratifiedSample(region=block_fc.geometry(), numPoints=max(3*count,20), scale=dem_resolution, projection=dwComposite.projection(), geometries=True)
    block_pts = block_pts.map(setPropertiesControl(block_fc))
    control_points_ee = control_points_ee.merge(block_pts)

  ## SAMPLE ON WHOLE AER ##
  # control_points_ee = composite_mask.stratifiedSample(region=aoi_boundary.geometry(), numPoints=no_sampling, scale=dem_resolution, projection=dwComposite.projection(), geometries=True)

  return control_points_ee, composite_mask

In [ ]:
seasonality = -1
control_points_ee, composite_mask = getControlGroup(dwComposite, aoi_boundary, None, treated_blocks, seasonality)
control_points_ee = control_points_ee.map(addPropertiesControls)
#print(control_points_ee.first().getInfo())

In [ ]:
print(control_points_ee.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [86.44752982090971, 23.877893988359972]}, 'id': '1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_2_0', 'properties': {'Block': 'govindpur', 'District': 'DHANBAD', 'State': 'JHARKHAND', 'Treatment': 0, 'label': 1, 'lat': 23.877893988359972, 'lon': 86.44752982090971}}


NEW METHOD TO EXPORT FILE: ENSURES CONTINOUS UPDATES ON EXPORT TASK AND CATCHES ERRORS (Export took 10 minutes for AEZ-12)

In [ ]:
def safe_export_to_drive(collection, folder, description, filename, file_format='CSV', selectors=None, max_retries=3):
    """
    Safely export Earth Engine FeatureCollection to Google Drive with error handling and retry logic.

    Args:
        collection (ee.FeatureCollection): The feature collection to export
        folder (str): Google Drive folder to export to
        description (str): Task description
        filename (str): Output filename
        file_format (str): Output file format (default: 'CSV')
        selectors (list): List of properties to include in the output (default: None)
        max_retries (int): Maximum number of retry attempts (default: 3)

    Returns:
        bool: True if export was successful, False otherwise
    """
    # Increase Python's recursion limit
    sys.setrecursionlimit(5000)

    def check_task_status(task):
        """Monitor task status"""
        status = task.status()['state']
        while status in ['READY', 'RUNNING']:
            print(f'Task status: {status}...')
            time.sleep(10)
            status = task.status()['state']
        return status

    for attempt in range(max_retries):
        try:
            # Create export task
            export_task = ee.batch.Export.table.toDrive(
                collection=collection,
                folder=folder,
                description=description,
                fileNamePrefix=filename,
                fileFormat=file_format,
                selectors=selectors
            )

            # Start the task
            export_task.start()

            # Monitor the task
            final_status = check_task_status(export_task)

            if final_status == 'COMPLETED':
                print(f'Export completed successfully after {attempt + 1} attempts')
                return True
            else:
                print(f'Export failed with status: {final_status}')
                if attempt < max_retries - 1:
                    print(f'Retrying... (Attempt {attempt + 2} of {max_retries})')
                    time.sleep(30)  # Wait before retrying
                continue

        except RecursionError:
            print(f'RecursionError encountered on attempt {attempt + 1}')
            if attempt < max_retries - 1:
                print('Reducing collection complexity and retrying...')
                # Optionally simplify the collection here if needed
                time.sleep(30)
            continue

        except Exception as e:
            print(f'Unexpected error on attempt {attempt + 1}: {str(e)}')
            if attempt < max_retries - 1:
                print(f'Retrying... (Attempt {attempt + 2} of {max_retries})')
                time.sleep(30)
            continue

    print('Export failed after all retry attempts')
    return False

✅Change the AEZ number

In [ ]:
# Your original export parameters
folder_run = 'DID-AEZ-12'
fn = "control_df"

# Use the safe export function
success = safe_export_to_drive(
    collection=control_points_ee,
    folder=folder_run,
    description=fn+folder_run,
    filename=fn,
    selectors=['lat','lon','Block','District','State','Treatment']
)

if success:
    print("Export completed successfully")
else:
    print("Export failed - please check the error messages above")

Task status: READY...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNING...
Task status: RUNNI

##Merge together treated and control. Add UIDs


✅Change the AEZ number

In [ ]:
folder_run = '/content/drive/MyDrive/Sanya/DID-AEZ-12/'
treated_df = pd.read_csv(folder_run+"treated_df.csv")
control_df = pd.read_csv(folder_run+"control_df.csv")

In [ ]:
# have to add these columns to control points. They are already set for treated points.
# We just use the string 'control' as Asset Name
control_df['Asset ID']=0
control_df['Asset Name']='control'
control_df['Total_Expenditure'] = 0
control_df

,lat,lon,Block,District,State,Treatment,Asset ID,Asset Name,Total_Expenditure
0,23.877894,86.447530,govindpur,DHANBAD,JHARKHAND,0,0,control,0
1,23.854178,86.486876,govindpur,DHANBAD,JHARKHAND,0,0,control,0
2,23.857951,86.522449,govindpur,DHANBAD,JHARKHAND,0,0,control,0
3,23.874660,86.449955,govindpur,DHANBAD,JHARKHAND,0,0,control,0
4,23.881128,86.498734,govindpur,DHANBAD,JHARKHAND,0,0,control,0
...,...,...,...,...,...,...,...,...,...
75262,19.474891,79.214026,wankdi,Kumram Bheem(Asifabad),TELANGANA,0,0,control,0
75263,19.485132,79.215373,wankdi,Kumram Bheem(Asifabad),TELANGANA,0,0,control,0
75264,19.479473,79.220763,wankdi,Kumram Bheem(Asifabad),TELANGANA,0,0,control,0
75265,19.498337,79.193544,wankdi,Kumram Bheem(Asifabad),TELANGANA,0,0,control,0


In [ ]:
# drop panchayat column
treated_df.drop(['Panchayat', 'idx'], axis=1, inplace=True,errors='ignore')

In [ ]:
# Set UIDs - Unique IDs. These help in identifying and concatenating data
treated_df['UID'] = range(1, len(treated_df)+1)
control_df['UID'] = range(len(treated_df)+1, len(control_df)+len(treated_df)+1)

In [ ]:
# reorder columns
cols = ['UID','lat', 'lon', 'Asset ID', 'Asset Name', 'Treatment','Block', 'District', 'State','Total_Expenditure']
treated_df = treated_df[cols]
control_df = control_df[cols]

In [ ]:
treated_df.reset_index(drop=True, inplace=True)
control_df.reset_index(drop=True, inplace=True)

In [ ]:
df = pd.concat([treated_df, control_df], ignore_index=True)
df

,UID,lat,lon,Asset ID,Asset Name,Treatment,Block,District,State,Total_Expenditure
0,1,18.099233,82.728270,2006446648,Farm Pond,1,Hukumpeta,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,42107.0
1,2,18.099442,82.732698,2006446650,Farm Pond,1,Hukumpeta,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,40670.0
2,3,18.099500,82.732721,2006446651,Farm Pond,1,Hukumpeta,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,40293.0
3,4,18.112828,82.693677,2005520196,Farm Pond,1,Hukumpeta,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,318782.0
4,5,18.139298,82.698729,2005520197,Farm Pond,1,Hukumpeta,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,228909.0
...,...,...,...,...,...,...,...,...,...,...
99872,99873,19.474891,79.214026,0,control,0,wankdi,Kumram Bheem(Asifabad),TELANGANA,0.0
99873,99874,19.485132,79.215373,0,control,0,wankdi,Kumram Bheem(Asifabad),TELANGANA,0.0
99874,99875,19.479473,79.220763,0,control,0,wankdi,Kumram Bheem(Asifabad),TELANGANA,0.0
99875,99876,19.498337,79.193544,0,control,0,wankdi,Kumram Bheem(Asifabad),TELANGANA,0.0


✅Change the AEZ number

In [ ]:
%cd /content/drive/MyDrive/Sanya/DID-AEZ-12/
df.to_csv("treated_and_control_df.csv", index=False)

/content/drive/MyDrive/Sanya/DID-AEZ-12


In [ ]:
treated_df.to_csv("treated_df_final.csv", index=False)
control_df.to_csv("control_df_final.csv", index=False)

# After exporting as csv to drive, you should download the CSV.
# Then manaully upload it to GEE as an asset. Ideally, create a folder with AEZ name in your working folder and uplaod there.
# Note -  while uploading, you need to set the latitude and longitude columns to the correct labels (lat and lon for our data). When uploading there is a dialog box where this can be put in.
# Left vbox for 'lon', right for 'lat'

##Sample 10000 points - 2500 treated and 7500 control

Deprecated - chunking method

In [ ]:
def addCovariate(var):
  def setProperty(f):
    return f.set({var:ee.Feature(ee.List(f.get('var')).get(0)).get(var)})
  return setProperty
eqFilter = ee.Filter.intersects(leftField= '.geo', rightField= '.geo')
saveAllJoin = ee.Join.saveAll(matchesKey= 'var')

In [ ]:
Treated = pd.read_csv("treated_df_final.csv")
Control = pd.read_csv("control_df_final.csv")

In [ ]:
confounding_vars = ['elevation',
                    'slope',
                    'dist_closest_crop',
                    'dist_closest_river',
                    'flow_accumulation',
                    'dist_closest_lin',
                    'dist_closest_road',
                    'recent_cropping_intensity',
                    'dist_closest_upstream_forest',
                    'proximity_water'
                    ]

In [ ]:
#randomly sample control points from them to be equal to 3 * treayed points
Treated_Sampled = Treated.sample(n=2500, random_state=42)
Control_Sampled = Control.sample(n=7500, random_state=42)

# Save the sampled DataFrame to a new CSV file
Treated_Sampled.to_csv('treated_sampled.csv', index=False)
Control_Sampled.to_csv('control_sampled.csv', index=False)

In [ ]:
# Combine into a single DataFrame
Combined_Sampled = pd.concat([Treated_Sampled, Control_Sampled], ignore_index=True)

# Save the combined DataFrame to a new CSV file
Combined_Sampled.to_csv('combined_sampled.csv', index=False)

##Find Covariates

In [ ]:
# Split Combined_Sampled DataFrame into 4 parts with 2500 points each
chunk_size = 2500
chunks = [Combined_Sampled[i:i + chunk_size] for i in range(0, len(Combined_Sampled), chunk_size)]

# Save each chunk to a separate CSV file
for idx, chunk in enumerate(chunks, start=1):
    # Define file name for each chunk
    file_name = f"covariates_sampled_points_part_{idx}.csv"
    # Save chunk to CSV
    chunk.to_csv(file_name, index=False)

# If needed, continue processing each chunk as an individual FeatureCollection in Google Earth Engine (GEE)
for idx, chunk in enumerate(chunks, start=1):
    data_points = geemap.pandas_to_ee(chunk, longitude='lon', latitude='lat')
    covariates_fc = data_points

    all_waterworks = ee.FeatureCollection("projects/ee-kshitiz/assets/waterworks_india_compact")
    aoi_assets_all_ww = all_waterworks.filterBounds(aoi_boundary_geometry)

    for var in confounding_vars:
        func = globals()['calc_' + var]
        var_fc = func(data_points, aoi_boundary, dwComposite, aoi_assets_all_ww)
        covariates_fc = saveAllJoin.apply(covariates_fc, var_fc, eqFilter)
        covariates_fc = covariates_fc.map(addCovariate(var))

    # Export each chunk individually
    fn = f"covariates_sampled_points_part_{idx}"
    export_task = ee.batch.Export.table.toDrive(
        collection=covariates_fc,
        folder=folder_run,
        description=f"description_{idx}",
        fileNamePrefix=fn,
        fileFormat='CSV'
    )
    export_task.start()

In [ ]:
# Get a list of all tasks
tasks = ee.batch.Task.list()

# Print the status of each task
for task in tasks:
    print(f"Task ID: {task.id}, Status: {task.state}, Description: {task.config.get('description', 'No description available')}")

Task ID: 5DB65W574WKLLS5MINZHXH3R, Status: CANCELLED, Description: Sentinel2_TimeSeriesBand_2021_AEZ298.csv
Task ID: L3MEGSNWXTYXJZPRUF5R53IS, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2023
Task ID: 7TEE6DT4OQDUMXXXKIPZOF3U, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2022
Task ID: FSW7LVXEQAEVTMIQYX2HDRZX, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2021
Task ID: 6RHQKRASZOLROVN4HEXAWEIA, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2020
Task ID: ICY2IDGDD35PGBEYJDEI5BOP, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2019
Task ID: CRUEE2T4V7YDABMXG3JSEBZH, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2018
Task ID: PDJ2NJO5CR54T5CKZ47PKJ76, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2017
Task ID: P65FDO6PBNLDA3FTBIKKMJV4, Status: COMPLETED, Description: AEZ_12_Watersheds_Precipitation_2016
Task ID: 6XOJTA3YI7Y7DYYYHW5YUAUW, Status: COMPLETED, Descri

In [ ]:
# List of file names for each part
file_parts = [
    "covariates_sampled_points_part_1.csv",
    "covariates_sampled_points_part_2.csv",
    "covariates_sampled_points_part_3.csv",
    "covariates_sampled_points_part_4.csv"
]

# Read each part into a DataFrame and concatenate them
df = pd.concat([pd.read_csv(file) for file in file_parts], ignore_index=True)

df.to_csv("covariates_sampled_points.csv", index=False)

#Step 3 - Compute Outcome Variables (NDMI, NDVI, GCI)

In [ ]:
scale_var = 10

In [ ]:
landsat_collection_name = "LANDSAT/LC08/C02/T1_L2"
# Landsat 8 has data from 2013
# Can use other versions, if you need to compute outcome variables for earlier years

In [ ]:
seasons_start_month =  {"Kharif": 7, "Rabi": 11,"Zaid": 3}
season_duration = {"Kharif": 4, "Rabi": 4, "Zaid": 4}
day = 1

In [ ]:
# This flag is to turn on union mask
# Union mask is a masking strategy that we use

use_union_masks = 1

In [ ]:
config_json = {"time_params":{}}

In [ ]:
# lulc_band refers to string label in the LULC rasters. It changes with different versions
lulc_band = "predicted_label"

In [ ]:

# These were updated to compute NDVI/NDMI/GCI for all years (for which it was possible), to check for parallel trends assumption
target_years = [int(i) for i in range(2016, 2023)]
print(target_years)
# print(base_year)
config_json['time_params']['base_year'] = 2015
base_year = 2015


# for normal analysis, use 2016 base year, and [2017, ...] as target years

[2016, 2017, 2018, 2019, 2020, 2021, 2022]


✅Change AEZ

In [ ]:
# to check LULC paths
config_json['lulc_assets'] = {
'2015': 'projects/ee-ictdiitd/assets/AEZ_12_2015-07-01_2016-06-30_LULCmap_10m',
'2016': 'projects/ee-ictdiitd/assets/AEZ_12_2016-07-01_2017-06-30_LULCmap_10m',
'2017': 'projects/ee-ictdiitd/assets/AEZ_12_2017-07-01_2018-06-30_LULCmap_10m',
'2018': 'projects/ee-ictdiitd/assets/AEZ_12_2018-07-01_2019-06-30_LULCmap_10m',
'2019': 'projects/ee-ictdiitd/assets/AEZ_12_2019-07-01_2020-06-30_LULCmap_10m',
'2020': 'projects/ee-ictdiitd/assets/AEZ_12_2020-07-01_2021-06-30_LULCmap_10m',
'2021': 'projects/ee-ictdiitd/assets/AEZ_12_2021-07-01_2022-06-30_LULCmap_10m',
'2022': 'projects/ee-ictdiitd/assets/AEZ_12_2022-07-01_2023-06-30_LULCmap_10m'
 }

In [ ]:
def get_crop_mask_composite_indsat(base_year, target_years):
  base_lulc = ee.Image(config_json['lulc_assets'][str(base_year)]).select(lulc_band).rename(str(base_year))
  base_lulc_mask = base_lulc.eq(8).Or(base_lulc.eq(9)).Or(base_lulc.eq(10)).Or(base_lulc.eq(11))
  lulc = base_lulc.updateMask(base_lulc_mask).remap([8,9,10,11], [1,2,2,3]).rename(str(base_year))
  for target_year in target_years:
    target_lulc = ee.Image(config_json['lulc_assets'][str(target_year)]).select(lulc_band).rename(str(target_year))
    target_lulc_mask = target_lulc.eq(8).Or(target_lulc.eq(9)).Or(target_lulc.eq(10)).Or(target_lulc.eq(11))
    target_lulc = target_lulc.updateMask(target_lulc_mask).remap([8,9,10,11], [1,2,2,3]).rename(str(target_year))
    lulc = lulc.addBands(target_lulc)
    print("Added band: "+ str(target_year))
  return lulc

In [ ]:
def get_water_mask_composite_indsat(base_year, target_years):
  base_lulc = ee.Image(config_json['lulc_assets'][str(base_year)]).select(lulc_band).rename(str(base_year))
  base_lulc_mask = base_lulc.neq(2).Or(base_lulc.neq(3)).Or(base_lulc.neq(4))
  lulc = base_lulc.updateMask(base_lulc_mask).rename(str(base_year))
  for target_year in target_years:
    target_lulc = ee.Image(config_json['lulc_assets'][str(target_year)]).select(lulc_band).rename(str(target_year))
    target_lulc_mask = target_lulc.neq(2).Or(target_lulc.neq(3)).Or(target_lulc.neq(4))
    target_lulc = target_lulc.updateMask(target_lulc_mask).rename(str(target_year))
    lulc = lulc.addBands(target_lulc)
    print("Added band: "+ str(target_year))
  return lulc

In [ ]:
# UNION MASK

# New masking strategy that we implemented
# First take all the cropping pixels. Any pixel which was labelled as cropping in *any* year (from our range), we consider it as a cropping pixel. Say this is mask1
# Then take all the water pixels. Any pixel which was labelled as water in *any* year (from our range), we consider it as a non-cropping/water pixel, which needs to be masked. Say this is mask2

# Then we mask out mask2 from mask1 to get the final mask.
def get_union_mask(base_year, target_years):
  base_lulc1 = ee.Image(config_json['lulc_assets'][str(base_year)]).select(lulc_band).rename(str(base_year))
  base_lulc_mask1 = base_lulc1.eq(8).Or(base_lulc1.eq(9)).Or(base_lulc1.eq(10)).Or(base_lulc1.eq(11))
  lulc1 = base_lulc1.updateMask(base_lulc_mask1).remap([8,9,10,11], [1,2,2,3]).rename(str(base_year))
  for target_year in target_years:
    target_lulc = ee.Image(config_json['lulc_assets'][str(target_year)]).select(lulc_band).rename(str(target_year))
    target_lulc_mask = target_lulc.eq(8).Or(target_lulc.eq(9)).Or(target_lulc.eq(10)).Or(target_lulc.eq(11))
    target_lulc = target_lulc.updateMask(target_lulc_mask).remap([8,9,10,11], [1,2,2,3]).rename(str(target_year))
    lulc1 = lulc1.Or(target_lulc)
    print("Added band: "+ str(target_year))
  # lulc1 is mask1 from comment above
#Even if water in one season - remove
  base_lulc2 = ee.Image(config_json['lulc_assets'][str(base_year)]).select(lulc_band).rename(str(base_year))
  base_lulc_mask2 = base_lulc2.neq(2).Or(base_lulc2.neq(3)).Or(base_lulc2.neq(4))
  lulc2 = base_lulc2.updateMask(base_lulc_mask2).rename(str(base_year))
  for target_year in target_years:
    target_lulc = ee.Image(config_json['lulc_assets'][str(target_year)]).select(lulc_band).rename(str(target_year))
    target_lulc_mask = target_lulc.neq(2).Or(target_lulc.neq(3)).Or(target_lulc.neq(4))
    target_lulc = target_lulc.updateMask(target_lulc_mask).rename(str(target_year))
    lulc2 = lulc2.updateMask(target_lulc)
    print("Added band: "+ str(target_year))
  # lulc2 is mask2 from comment above

  lulc_new = lulc1.updateMask(lulc2)
  # create final mask
  return lulc_new

In [ ]:
# Computes NDVI and NDMI from the available spectral bands

def image_collection_reducer(imageCollection, reducer=ee.Reducer.mean(), scale= scale_var, crs='EPSG:4326', bestEffort=True, maxPixels=1e13, tileScale=4):
  return imageCollection.reduce(reducer)

def getNDVI_Landsat(image):
  return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI') # Landsat 8 bands

def getNDMI_Landsat(image):
  return image.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDMI') # Landsat 8 bands

def addVIBands_Landsat(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0)
    ndvi = getNDVI_Landsat(image)
    image = image.addBands(ndvi)
    image.updateMask(qaMask).updateMask(saturationMask)
    return(image)

def addMIBands_Landsat(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0)
    ndmi = getNDMI_Landsat(image)
    image = image.addBands(ndmi)
    image.updateMask(qaMask).updateMask(saturationMask)
    return(image)

In [ ]:
# Computes GCI from the available spectral bands

def image_collection_reducer(imageCollection, reducer=ee.Reducer.mean(), scale= scale_var, crs='EPSG:4326', bestEffort=True, maxPixels=1e13, tileScale=4):
  return imageCollection.reduce(reducer)

def getGCI_Landsat(image):
    return image.expression('(((NIR) / (GREEN)) - 1)', {'NIR': image.select('SR_B5'), 'GREEN': image.select('SR_B3')}).rename('GCI')
    # return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI') # Landsat 8 bands

def addGCIBands_Landsat(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0)
    ndvi = getGCI_Landsat(image)
    image = image.addBands(ndvi)
    image.updateMask(qaMask).updateMask(saturationMask)
    return(image)


In [ ]:
def compute_mean_over_mean_seasonal_NDMI(feature, season, year):
  season_start = ee.Date.fromYMD(year + 1, seasons_start_month[season], day) if season == 'Zaid' else ee.Date.fromYMD(year, seasons_start_month[season], day)
  season_end = season_start.advance(season_duration[season], 'month')
  feature_bufd = feature.buffer(100).geometry();
  lsat_kharif_collection = ee.ImageCollection(landsat_collection_name).filterBounds(feature_bufd).filterDate(season_start, season_end)
  vi_kharif_collection = lsat_kharif_collection.map(addMIBands_Landsat).select('NDMI');
  vi_kharif_composite = image_collection_reducer(vi_kharif_collection, ee.Reducer.mean())
  if use_union_masks:
    ndvi_kharif_composite = vi_kharif_composite.select('NDMI_mean').updateMask(lulc_new)
  else:
    ndvi_kharif_composite = vi_kharif_composite.select('NDMI_mean').updateMask(lulc.select(str(year)))
  ndvi_kharif = ndvi_kharif_composite.reduceRegion(ee.Reducer.mean(), feature_bufd, scale=scale_var, crs='EPSG:4326')
  return feature.set('ndmi_'+str(season)+'_'+str(year), ndvi_kharif.get('NDMI_mean'))

def compute_mean_over_mean_seasonal_NDVI(feature, season, year):
  season_start = ee.Date.fromYMD(year + 1, seasons_start_month[season], day) if season == 'Zaid' else ee.Date.fromYMD(year, seasons_start_month[season], day)
  season_end = season_start.advance(season_duration[season], 'month')
  feature_bufd = feature.buffer(100).geometry();
  lsat_kharif_collection = ee.ImageCollection(landsat_collection_name).filterBounds(feature_bufd).filterDate(season_start, season_end)
  vi_kharif_collection = lsat_kharif_collection.map(addVIBands_Landsat).select('NDVI');
  vi_kharif_composite = image_collection_reducer(vi_kharif_collection, ee.Reducer.mean())
  if use_union_masks:
    ndvi_kharif_composite = vi_kharif_composite.select('NDVI_mean').updateMask(lulc_new)
  else:
    ndvi_kharif_composite = vi_kharif_composite.select('NDVI_mean').updateMask(lulc.select(str(year)))
  ndvi_kharif = ndvi_kharif_composite.reduceRegion(ee.Reducer.mean(), feature_bufd, scale=scale_var, crs='EPSG:4326')
  return feature.set('ndvi_'+str(season)+'_'+str(year), ndvi_kharif.get('NDVI_mean'))

def compute_mean_over_mean_seasonal_GCI(feature, season, year):
  season_start = ee.Date.fromYMD(year + 1, seasons_start_month[season], day) if season == 'Zaid' else ee.Date.fromYMD(year, seasons_start_month[season], day)
  season_end = season_start.advance(season_duration[season], 'month')
  feature_bufd = feature.buffer(100).geometry();
  lsat_kharif_collection = ee.ImageCollection(landsat_collection_name).filterBounds(feature_bufd).filterDate(season_start, season_end)
  vi_kharif_collection = lsat_kharif_collection.map(addGCIBands_Landsat).select('GCI');

  vi_kharif_composite = image_collection_reducer(vi_kharif_collection, ee.Reducer.mean())
  if use_union_masks:
    GCI_kharif_composite = vi_kharif_composite.select('GCI_mean').updateMask(lulc_new)
  else:
    GCI_kharif_composite = vi_kharif_composite.select('GCI_mean').updateMask(lulc.select(str(year)))
  GCI_kharif = GCI_kharif_composite.reduceRegion(ee.Reducer.mean(), feature_bufd, scale=scale_var, crs='EPSG:4326')
  return feature.set('GCI_'+str(season)+'_'+str(year), GCI_kharif.get('GCI_mean'))

In [ ]:
### USE THIS-
### set data_points to your asset
data_points = ee.FeatureCollection('projects/ee-sanyakapoor/assets/AEZ-12/combined_sampled')

In [ ]:
print(data_points.first().getInfo());
# This helps check if an asset is present. Always do this before running any tasks

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [82.11236605833813, 17.64597384419506]}, 'id': '0000000000000000009a', 'properties': {'Asset ID': 2006232785, 'Asset Name': 'Farm Pond', 'Block': 'Addateegala', 'District': 'ALLURI SITHARAMA RAJU', 'State': 'ANDHRA PRADESH', 'Total_Expenditure': 68317, 'Treatment': 1, 'UID': 284}}


In [ ]:
# years = [2016, 2017, 2018, 2019, 2020, 2021, 2022]
years = target_years.copy()
seasons = ['Kharif', 'Rabi', 'Zaid']

In [ ]:
## MASKING STRATEGY
## Here you choose if you want to use cropmask, watermask or unionmask

# For unionmask, leave lulc = get_crop_mask_composite_indsat(.) and set union_mask flag as 1.

# lulc = get_water_mask_composite_indsat(base_year, target_years)
lulc = get_crop_mask_composite_indsat(base_year, target_years)


# list17_22 = [i for i in range(2017, 2023)]
list11_22 = target_years.copy()


# This give the union mask
lulc_new = get_union_mask(2015, list11_22)

Added band: 2016
Added band: 2017
Added band: 2018
Added band: 2019
Added band: 2020
Added band: 2021
Added band: 2022
Added band: 2016
Added band: 2017
Added band: 2018
Added band: 2019
Added band: 2020
Added band: 2021
Added band: 2022
Added band: 2016
Added band: 2017
Added band: 2018
Added band: 2019
Added band: 2020
Added band: 2021
Added band: 2022


In [ ]:
## To check the created mask
lulc.getInfo()

{'type': 'Image',
 'bands': [{'id': '2015',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 3},
   'dimensions': [102486, 93891],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    78.7721892860999,
    0,
    -8.983152841195215e-05,
    25.34884035034148]},
  {'id': '2016',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 3},
   'dimensions': [102486, 93891],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    78.7721892860999,
    0,
    -8.983152841195215e-05,
    25.34884035034148]},
  {'id': '2017',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 3},
   'dimensions': [102486, 93891],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    78.7721892860999,
    0,
    -8.983152841195215e-05,
    25.34884035034148]},
  {'id': '2018',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 1, 'max': 3},
   'dimensions': [1

In [ ]:
for year in years:
  for season in seasons:
    data_points = data_points.map(lambda f: compute_mean_over_mean_seasonal_NDVI(f, season, year))
  print("NDVI completed for year: "+ str(year))

for year in years:
  for season in seasons:
    data_points = data_points.map(lambda f: compute_mean_over_mean_seasonal_NDMI(f, season, year))
  print("NDMI completed for year: "+ str(year))

for year in years:
  for season in seasons:
    data_points = data_points.map(lambda f: compute_mean_over_mean_seasonal_GCI(f, season, year))
  print("GCI completed for year: "+ str(year))

NDVI completed for year: 2016
NDVI completed for year: 2017
NDVI completed for year: 2018
NDVI completed for year: 2019
NDVI completed for year: 2020
NDVI completed for year: 2021
NDVI completed for year: 2022
NDMI completed for year: 2016
NDMI completed for year: 2017
NDMI completed for year: 2018
NDMI completed for year: 2019
NDMI completed for year: 2020
NDMI completed for year: 2021
NDMI completed for year: 2022
GCI completed for year: 2016
GCI completed for year: 2017
GCI completed for year: 2018
GCI completed for year: 2019
GCI completed for year: 2020
GCI completed for year: 2021
GCI completed for year: 2022


In [ ]:
data_points.size()

✅Change AEZ

In [ ]:
output_prefix_path = 'users/mtpictd/NDVI/'
fc_export_task2 = ee.batch.Export.table.toAsset(
    collection = data_points,
    description = 'mean_of_mean_unionmask_AEZ12_sampled_points',
    assetId = output_prefix_path + 'mean_of_mean_unionmask_AEZ12_2016-2023_Landsat_v2'
  )
fc_export_task2.start()

# ENDS HERE

#Step 4 - Some more covariates

Soil based - pH, OC, CEC, HSG [Took 4 mins]
	Location - users/mtpictd/Kshitiz/All_Soil_Covariates (GEE)
	Simply import your asset for points and run. The file will be exported to the specified location in google drive. We will collate it manually to the other data.
Run one by one for all covariates






##Drainage Density

In [ ]:
DrainageImage = ee.Image('projects/ee-priyadarshini/assets/india_drainage_density')

In [ ]:
df= pd.read_csv("combined_sampled.csv")
df

,UID,lat,lon,Asset ID,Asset Name,Treatment,Block,District,State,Total_Expenditure
0,3734,24.391782,86.647062,34001162850,Dova nirman,1,deoghar,DEOGHAR,JHARKHAND,17869.0
1,11503,22.683443,86.417664,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,1,ghatshila,EAST SINGHBUM,JHARKHAND,0.0
2,6424,23.960886,86.472827,34001109193,DOBHA CONT. OF BABUJAN SOREN,1,tundi,DHANBAD,JHARKHAND,0.0
3,15984,24.066874,85.733054,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,1,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0
4,23614,22.599589,85.430967,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,1,sonua,WEST SINGHBHUM,JHARKHAND,7560.0
...,...,...,...,...,...,...,...,...,...,...
9995,93929,21.481009,85.008159,0,control,0,barkote,DEOGARH,ODISHA,0.0
9996,80212,22.360100,85.155573,0,control,0,manoharpur,WEST SINGHBHUM,JHARKHAND,0.0
9997,72004,18.537050,83.362625,0,control,0,Bobbili,VIZIANAGARAM,ANDHRA PRADESH,0.0
9998,36536,24.419039,87.009157,0,control,0,jarmundi,DUMKA,JHARKHAND,0.0


In [ ]:
# takes ~10mins for 7k points
data={}
for i in df.index:
  # print(i)
  lon=df['lon'][i]
  lat=df['lat'][i]
  p = ee.Geometry.Point([lon, lat])
  data_b1 = DrainageImage.select("b1").reduceRegion(ee.Reducer.mean(),p,100).get("b1")
  data[i]=data_b1.getInfo()
print(data)
new_column = pd.Series(data)
df['drainage_density'] = new_column
df.head()

{0: 0.007047516293823719, 1: 0.006092604715377092, 2: 0.0060460832901299, 3: 0.006410655565559864, 4: 0.006100130267441273, 5: 0.006527242716401815, 6: 0.005736060440540314, 7: 0.0055769351311028, 8: 0.006340589374303818, 9: 0.006965982727706432, 10: 0.0029850138816982508, 11: 0.005383409559726715, 12: 0.005852524656802416, 13: 0.005750434473156929, 14: 0.005752879660576582, 15: 0.005770829971879721, 16: 0.005164758302271366, 17: 0.0054392702877521515, 18: 0.00552879273891449, 19: 0.005942097399383783, 20: 0.006596458610147238, 21: 0.006200691219419241, 22: 0.006061308551579714, 23: 0.006395986303687096, 24: 0.0061935014091432095, 25: 0.0061729843728244305, 26: 0.005626796279102564, 27: 0.006283234339207411, 28: 0.005450560711324215, 29: 0.005898552481085062, 30: 0.005898542236536741, 31: 0.0056992582976818085, 32: 0.006182795390486717, 33: 0.00476812245324254, 34: 0.005505038425326347, 35: 0.005985418800264597, 36: 0.0047326176427304745, 37: 0.0057370043359696865, 38: 0.00586523301899

,UID,lat,lon,Asset ID,Asset Name,Treatment,Block,District,State,Total_Expenditure,drainage_density
0,3734,24.391782,86.647062,34001162850,Dova nirman,1,deoghar,DEOGHAR,JHARKHAND,17869.0,0.007048
1,11503,22.683443,86.417664,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,1,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,0.006093
2,6424,23.960886,86.472827,34001109193,DOBHA CONT. OF BABUJAN SOREN,1,tundi,DHANBAD,JHARKHAND,0.0,0.006046
3,15984,24.066874,85.733054,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,1,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,0.006411
4,23614,22.599589,85.430967,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,1,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,0.006100


✅Change AEZ

In [ ]:
df.to_csv("AEZ12_ddensity.csv", index=False)

##Precipipation

Precipitation is computed at watershed level. In this script we first mark the watersheds that belong to our area of interest, then match each point to a WS (in which the point lies), and then compute monthly precipitation for each WS.
Change the paths for export tasks and run.

In [ ]:
WaterSheds = ee.FeatureCollection("users/mtpictd/watersheds_india")
geometry = ee.Geometry.MultiPoint()

✅Change AEZ

In [ ]:
# Change here
area_of_interest = ee.FeatureCollection('users/mtpictd/agro_eco_regions').filter(ee.Filter.eq('ae_regcode', 12))

In [ ]:
AEZs = [area_of_interest, ]
AEZ_watersheds = []  # To store group of watersheds of each AEZ respectively

for AEZ in AEZs:  # for iteration in each AEZ
  AEZGeometry = AEZ.geometry() # getting the geometry of AEZ
  IntersectedWaterSheds = WaterSheds.filter(ee.Filter.intersects(".geo", AEZGeometry))  # Intersected watersheds
  def check_area_percentage(feature):
      inner_geometry = feature.geometry() # Watershed geometry
      inner_area = inner_geometry.area() # Watershed area
      intersection_area = inner_geometry.intersection(AEZGeometry).area() # Intersected area with AEZ
      percentage_contained = intersection_area.divide(inner_area)
      is_more_than_0_percent = percentage_contained.gte(0)         ### Greater then 1 for complete inside, 0.5 for >50% area, 0 for intersection
      return feature.set('containsMoreThan0Percent', is_more_than_0_percent) # set 1 if intersected area > 0%
  result_collection = IntersectedWaterSheds.map(check_area_percentage)
  # Filter out the features having containsMoreThan0Percent as true
  watershed_collection = result_collection.filterMetadata('containsMoreThan0Percent', 'equals', 1)
  # Append the watershed collection to the list
  AEZ_watersheds.append(watershed_collection)

# # Loop through the district_watersheds list and print each feature collection

for i, watershed_collection in enumerate(AEZ_watersheds):  # To print Watersheds
    # print(f"District {district_names[i]} Watersheds:")
    # pp = pprint.PrettyPrinter(indent=2)
    # pp.pprint(watershed_collection.limit(1).getInfo())
    print(watershed_collection.size().getInfo())
    # print()
    # pp.pprint(watershed_collection.getInfo())


for i, watershed_collection in enumerate(AEZ_watersheds):
  # Define the asset ID where you want to upload the feature collection
  asset_id = 'projects/ee-sanyakapoor/assets/' + 'AEZ12_watersheds'

  # Export the feature collection to the asset
  task = ee.batch.Export.table.toAsset(
      collection=watershed_collection,  # Flatten the nested list of feature collections
      description= 'AEZ12_Watersheds_export',  # Description for the export task
      assetId=asset_id  # The asset ID where you want to upload the feature collection
  )
  task.start()
  ee.batch.Export.table.toDrive(
        collection=watershed_collection,
        description= 'AEZ_12_Watersheds',
        fileFormat='CSV',
        folder='DID-AEZ-12'
    ).start()



535


✅Change AEZ

In [ ]:
points = ee.FeatureCollection('projects/ee-sanyakapoor/assets/AEZ-12/combined_sampled')

In [ ]:
points.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [82.11236605833813, 17.64597384419506]},
 'id': '0000000000000000009a',
 'properties': {'Asset ID': 2006232785,
  'Asset Name': 'Farm Pond',
  'Block': 'Addateegala',
  'District': 'ALLURI SITHARAMA RAJU',
  'State': 'ANDHRA PRADESH',
  'Total_Expenditure': 68317,
  'Treatment': 1,
  'UID': 284}}

✅Change AEZ

In [ ]:
WaterSheds = ee.FeatureCollection('projects/ee-sanyakapoor/assets/AEZ12_watersheds')

In [ ]:
WaterSheds.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[82.22468791820839, 18.77127586270863],
    [82.22551510234648, 18.771263464124935],
    [82.22550200015692, 18.77047553989392],
    [82.22633398437291, 18.770463066033294],
    [82.2263207994109, 18.769670548107435],
    [82.22714796443202, 18.76965813896809],
    [82.22713486254526, 18.768870215985814],
    [82.22796201803487, 18.76885780158456],
    [82.2279358974833, 18.76728196200505],
    [82.22876304359337, 18.767269541275336],
    [82.22874994200616, 18.766481621231396],
    [82.23040420547514, 18.766456675039077],
    [82.23039102127284, 18.765664250278306],
    [82.23122294790738, 18.765651744629025],
    [82.2312098467765, 18.764863824773336],
    [82.2328640526263, 18.764838849091912],
    [82.23283785097102, 18.763263100211194],
    [82.23366493967075, 18.763250558940722],
    [82.23365183899293, 18.762462730982456],
    [82.23282475013974, 18.76247518330208],
    [82.23281164930602, 18.761687268086657

In [ ]:
# Create a spatial filter to perform the spatial join
spatialFilter = ee.Filter.intersects(
    leftField='.geo',
    rightField='.geo',
    maxError=10
)

spatialJoined = ee.Join.saveFirst(matchKey='pointInRegion').apply(points, WaterSheds, spatialFilter)

def assign_val(point):
    pointWithRegion = ee.Feature(point.get('pointInRegion'))
    point = point.set('objectid', pointWithRegion.get('objectid'))
    return point

# Map the function to assign wscode
FarmPondsWithWsCode = spatialJoined.map(assign_val)

# Set 'pointInRegion' property to null
FarmPondsWithWsCode = FarmPondsWithWsCode.map(lambda feature: feature.set('pointInRegion', None))

In [ ]:
FarmPondsWithWsCode.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [82.11236605833813, 17.64597384419506]},
 'id': '0000000000000000009a',
 'properties': {'Asset ID': 2006232785,
  'Asset Name': 'Farm Pond',
  'Block': 'Addateegala',
  'District': 'ALLURI SITHARAMA RAJU',
  'State': 'ANDHRA PRADESH',
  'Total_Expenditure': 68317,
  'Treatment': 1,
  'UID': 284,
  'objectid': 1417}}

✅Change AEZ

In [ ]:
task = ee.batch.Export.table.toAsset(
    collection= FarmPondsWithWsCode,
    description= f'AEZ12_all_points_with_ws_objectid',
    assetId='projects/ee-sanyakapoor/assets/AEZ-12_all_points_with_ws_objectid'
)
task.start()
ee.batch.Export.table.toDrive(
        collection=FarmPondsWithWsCode,
        description= 'AEZ12_all_points_with_ws_objectid',
        fileFormat='CSV',
        folder='DID-AEZ-12'
    ).start()

TAKES 5.5 HOURS FOR EACH YEAR!

In [ ]:
# Define a function to calculate monthly precipitation and set it as properties
def calculate_monthly_precipitation(roi_feature, start, end): # roi_feature :- Feature collection, start :- start date, end :- end date
    # Define the start and end dates for your analysis
    start_date = ee.Date(start)
    end_date = ee.Date(end)
    for year in range(start_date.get('year').getInfo(), end_date.get('year').getInfo() + 1):
      # print("hi")
      for month in range(1, 13):
        # print("hi1")
        start = ee.Date.fromYMD(year, month, 1)
        end = start.advance(1, 'month')
        monthly_precipitation = ee.ImageCollection('JAXA/GPM_L3/GSMaP/v6/operational') \
            .filterBounds(roi_feature.geometry()) \
            .filterDate(start, end) \
            .sum() \
            .clip(roi_feature.geometry())

        # Calculate the mean precipitation for the month
        mean_precipitation = monthly_precipitation.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi_feature.geometry(),
            scale=1000,
            maxPixels=1e9,
            bestEffort=True
        ).get('hourlyPrecipRate')
        # print(mean_precipitation.getInfo())
        # Set the precipitation as a property of the feature
        up = str(month)
        if month < 10:
          up = '0' + up
        property_name = 'year_' + str(year) + '_month_' + up
        roi_feature = roi_feature.set(property_name, mean_precipitation)
    return roi_feature

for k in range(len(AEZ_watersheds)):
  WatershedCollection = AEZ_watersheds[k]
  StartYear = 2017
  # TO DO - clean up the code
  for i in range(9):
    # 9 is for range till 2023
    Year = StartYear + i
    start = str(Year) + '-01-01'
    end = str(Year) + '-12-31'
    # start = '2021-01-01'
    # end = '2022-12-31'
    Water = WatershedCollection.map(lambda feature: calculate_monthly_precipitation(feature, start, end))

    # Export the feature collection to Google Drive as a CSV file
    ee.batch.Export.table.toDrive(
        collection=Water,
        description= 'AEZ_12_Watersheds_Precipitation_' + str(Year),
        fileFormat='CSV',
        folder='DID-AEZ-12'
    ).start()


# Map the function to the feature collection
# roi_collection_with_precipitation = roi_collection.map(calculate_monthly_precipitation)

# # Print the feature collection with monthly precipitation properties
# print(roi_collection_with_precipitation.getInfo())




In [ ]:
new_df = pd.read_csv('combined_sampled.csv')

✅Change AEZ

In [ ]:
all_data = pd.DataFrame()
notset = 0
for year in range(2015, 2023):
    file_name = f"AEZ_12_Watersheds_Precipitation_{year}.csv"
    data = pd.read_csv(file_name)
    print(data.shape)
    if notset == 0:
        notset = 1
        all_data = data[['objectid', 'wscode']].copy()
    cols = list(data.columns)
    keep_cols = [col for col in cols if 'year' in col]
    keep_cols.append('objectid')
    keep_data = data[keep_cols]
    all_data = pd.merge(all_data, keep_data, on='objectid')

(535, 23)
(535, 23)
(535, 23)
(535, 23)
(535, 23)
(535, 23)
(535, 23)
(535, 23)


In [ ]:
all_cols = list(all_data.columns)
all_cols
global_data = all_data.copy()
for col in all_cols:
    if 'year' in col:
        col_split = col.split('_')
        yr = int(col_split[1])
        mth = int(col_split[3])
        new_col = f'Precipitation-{yr}-{mth}'
        global_data.rename(columns={col:new_col}, inplace=True)

In [ ]:
global_data.to_csv('AEZ12_Precipitation_data_watershed_lvl_global.csv', index=False)

In [ ]:
global_data

,objectid,wscode,Precipitation-2015-1,Precipitation-2015-2,Precipitation-2015-3,Precipitation-2015-4,Precipitation-2015-5,Precipitation-2015-6,Precipitation-2015-7,Precipitation-2015-8,Precipitation-2015-9,Precipitation-2015-10,Precipitation-2015-11,Precipitation-2015-12,Precipitation-2016-1,Precipitation-2016-2,Precipitation-2016-3,Precipitation-2016-4,Precipitation-2016-5,Precipitation-2016-6,Precipitation-2016-7,Precipitation-2016-8,Precipitation-2016-9,Precipitation-2016-10,Precipitation-2016-11,Precipitation-2016-12,Precipitation-2017-1,Precipitation-2017-2,Precipitation-2017-3,Precipitation-2017-4,Precipitation-2017-5,Precipitation-2017-6,Precipitation-2017-7,Precipitation-2017-8,Precipitation-2017-9,Precipitation-2017-10,Precipitation-2017-11,Precipitation-2017-12,Precipitation-2018-1,Precipitation-2018-2,Precipitation-2018-3,Precipitation-2018-4,Precipitation-2018-5,Precipitation-2018-6,Precipitation-2018-7,Precipitation-2018-8,Precipitation-2018-9,Precipitation-2018-10,Precipitation-2018-11,Precipitation-2018-12,Precipitation-2019-1,Precipitation-2019-2,Precipitation-2019-3,Precipitation-2019-4,Precipitation-2019-5,Precipitation-2019-6,Precipitation-2019-7,Precipitation-2019-8,Precipitation-2019-9,Precipitation-2019-10,Precipitation-2019-11,Precipitation-2019-12,Precipitation-2020-1,Precipitation-2020-2,Precipitation-2020-3,Precipitation-2020-4,Precipitation-2020-5,Precipitation-2020-6,Precipitation-2020-7,Precipitation-2020-8,Precipitation-2020-9,Precipitation-2020-10,Precipitation-2020-11,Precipitation-2020-12,Precipitation-2021-1,Precipitation-2021-2,Precipitation-2021-3,Precipitation-2021-4,Precipitation-2021-5,Precipitation-2021-6,Precipitation-2021-7,Precipitation-2021-8,Precipitation-2021-9,Precipitation-2021-10,Precipitation-2021-11,Precipitation-2021-12,Precipitation-2022-1,Precipitation-2022-2,Precipitation-2022-3,Precipitation-2022-4,Precipitation-2022-5,Precipitation-2022-6,Precipitation-2022-7,Precipitation-2022-8,Precipitation-2022-9,Precipitation-2022-10,Precipitation-2022-11,Precipitation-2022-12
0,192,5,0.003834,0.550387,5.082069,65.803309,58.756237,331.363581,144.265010,200.923438,321.831072,5.967952,66.051390,1.487812,0.437550,4.185952,7.578084,5.558516,118.094840,326.875543,300.257543,191.380255,199.027963,45.658892,0.834103,0.164918,0.467179,1.450454,10.843294,32.389322,69.046593,283.110658,350.577825,315.621595,234.894204,142.444501,0.239528,0.007639,0.0,7.504649,12.380115,55.640872,95.075274,149.086012,346.790972,378.418623,206.784349,12.973261,0.000000,13.866425,0.622451,0.790667,0.314554,64.890685,115.941424,221.523799,260.268994,430.306608,169.897550,172.061145,0.754757,3.103795,0.642169,0.519000,6.845218,70.312046,78.851460,223.248604,213.594654,271.675313,145.593088,152.494077,21.180584,0.007927,0.076923,0.781144,0.743791,86.097007,124.836533,120.424765,184.815590,146.947631,314.990318,57.831882,42.604040,2.376078,3.225056,0.002116,0.246613,53.420760,109.768849,144.555950,245.908842,390.427161,252.799142,74.234774,0.970670,4.480517
1,196,4,0.206668,0.664164,7.419349,75.565507,60.379801,337.034377,121.729222,196.769170,327.077399,6.112789,65.938583,1.493128,0.782911,4.063181,8.115781,6.099972,127.808724,308.827519,311.268555,186.458083,190.662510,46.298615,0.265719,1.477524,0.963699,1.252353,8.651418,29.017994,70.642334,307.882414,328.905815,324.192953,256.710149,156.204439,0.262790,0.269553,0.0,8.405787,17.999446,56.846241,104.129762,160.668741,359.272373,375.547089,228.947541,23.257678,0.000000,21.876811,0.402341,0.407817,0.288766,60.414336,111.173919,232.704190,265.353267,406.046646,161.867772,197.537026,0.502737,3.103567,1.009763,0.528110,9.480315,76.831785,80.854971,232.508735,198.834367,264.604949,142.935078,139.247141,23.000543,0.003575,0.051460,2.200503,0.677333,83.770378,121.151764,131.482472,181.685880,147.726161,318.429789,56.933904,39.131035,8.250818,4.367338,0.000000,0.273162,47.322207,114.369673,162.731830,228.957345,355.412958,236.551058,91.756330,1.990467,4.642181
2,212,3,0.5

#Step 5 - Collate covariates

Now we collate the new covariates with the old static covariates. Namely, solid based (pH, OC, CEC, HSG) and drainage density are collated.
It also has a section to add watershed objectids to the observation points.




In [ ]:
hsg = pd.read_csv("All_soil_covariates_HSG_CEC_OC_pH.csv")

In [ ]:
ph = hsg[['UID', 'pH']]
oc = hsg[['UID', 'OC']]
cec = hsg[['UID', 'CEC']]
hsg = hsg[['UID', 'HSG']]

In [ ]:
# Here, you read the static covariates file. This is the output file of the find_counterfactuals script.

# %cd /content/drive/MyDrive/Kshitiz_Bansal/AEZ6

points = pd.read_csv("combined_sampled.csv")
points.shape

(10000, 10)

In [ ]:
print(points.duplicated(subset='UID').sum())

0


In [ ]:
hsg.shape

(10000, 2)

In [ ]:
print(points.columns)
print(hsg.columns)
print(ph.columns)
print(oc.columns)
print(cec.columns)

Index(['UID', 'lat', 'lon', 'Asset ID', 'Asset Name', 'Treatment', 'Block',
       'District', 'State', 'Total_Expenditure'],
      dtype='object')
Index(['UID', 'HSG'], dtype='object')
Index(['UID', 'pH'], dtype='object')
Index(['UID', 'OC'], dtype='object')
Index(['UID', 'CEC'], dtype='object')


In [ ]:
points

,UID,lat,lon,Asset ID,Asset Name,Treatment,Block,District,State,Total_Expenditure
0,3734,24.391782,86.647062,34001162850,Dova nirman,1,deoghar,DEOGHAR,JHARKHAND,17869.0
1,11503,22.683443,86.417664,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,1,ghatshila,EAST SINGHBUM,JHARKHAND,0.0
2,6424,23.960886,86.472827,34001109193,DOBHA CONT. OF BABUJAN SOREN,1,tundi,DHANBAD,JHARKHAND,0.0
3,15984,24.066874,85.733054,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,1,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0
4,23614,22.599589,85.430967,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,1,sonua,WEST SINGHBHUM,JHARKHAND,7560.0
...,...,...,...,...,...,...,...,...,...,...
9995,93929,21.481009,85.008159,0,control,0,barkote,DEOGARH,ODISHA,0.0
9996,80212,22.360100,85.155573,0,control,0,manoharpur,WEST SINGHBHUM,JHARKHAND,0.0
9997,72004,18.537050,83.362625,0,control,0,Bobbili,VIZIANAGARAM,ANDHRA PRADESH,0.0
9998,36536,24.419039,87.009157,0,control,0,jarmundi,DUMKA,JHARKHAND,0.0


In [ ]:
hsg = hsg.drop_duplicates(subset='UID', keep='first')
ph = ph.drop_duplicates(subset='UID', keep='first')
oc = oc.drop_duplicates(subset='UID', keep='first')
cec = cec.drop_duplicates(subset='UID', keep='first')

In [ ]:
print(hsg[hsg.duplicated(subset='UID', keep=False)])
print(ph[ph.duplicated(subset='UID', keep=False)])
print(oc[oc.duplicated(subset='UID', keep=False)])
print(cec[cec.duplicated(subset='UID', keep=False)])

Empty DataFrame
Columns: [UID, HSG]
Index: []
Empty DataFrame
Columns: [UID, pH]
Index: []
Empty DataFrame
Columns: [UID, OC]
Index: []
Empty DataFrame
Columns: [UID, CEC]
Index: []


In [ ]:
print(hsg.duplicated(subset='UID').sum())  # Number of duplicate UIDs in hsg
print(ph.duplicated(subset='UID').sum())   # Number of duplicate UIDs in ph
print(oc.duplicated(subset='UID').sum())   # Number of duplicate UIDs in oc
print(cec.duplicated(subset='UID').sum())
print(points.duplicated(subset='UID').sum())

0
0
0
0
0


In [ ]:
# Merge the soil based covaraites with input data. Note, now you will only need to do 1 merge when the soil covars are in 1 dataframe.
points = pd.merge(points, hsg, on='UID')
points = pd.merge(points, ph, on='UID')
points = pd.merge(points, oc, on='UID')
points = pd.merge(points, cec, on='UID')
points

,UID,lat,lon,Asset ID,Asset Name,Treatment,Block,District,State,Total_Expenditure,HSG,pH,OC,CEC
0,3734,24.391782,86.647062,34001162850,Dova nirman,1,deoghar,DEOGHAR,JHARKHAND,17869.0,13.0,6.2,0.58,15.0
1,11503,22.683443,86.417664,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,1,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,3.0,7.9,0.75,43.0
2,6424,23.960886,86.472827,34001109193,DOBHA CONT. OF BABUJAN SOREN,1,tundi,DHANBAD,JHARKHAND,0.0,3.0,7.6,1.40,0.0
3,15984,24.066874,85.733054,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,1,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,3.0,6.2,0.58,15.0
4,23614,22.599589,85.430967,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,1,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,13.0,4.9,0.99,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,93929,21.481009,85.008159,0,control,0,barkote,DEOGARH,ODISHA,0.0,3.0,6.3,0.53,6.0
9996,80212,22.360100,85.155573,0,control,0,manoharpur,WEST SINGHBHUM,JHARKHAND,0.0,3.0,7.6,1.40,0.0
9997,72004,18.537050,83.362625,0,control,0,Bobbili,VIZIANAGARAM,ANDHRA PRADESH,0.0,14.0,6.2,0.58,15.0
9998,36536,24.419039,87.009157,0,control,0,jarmundi,DUMKA,JHARKHAND,0.0,3.0,6.2,0.58,15.0


In [ ]:
print(points.duplicated(subset='UID').sum())

0


✅Change AEZ

In [ ]:
# merge drainage density too
# Read the corresponding drainage densitey file and merge
# %cd AEZ2

dd = pd.read_csv("AEZ12_ddensity.csv")
dd = dd[['UID', 'drainage_density']]
dd = dd.drop_duplicates(subset='UID', keep='first')

In [ ]:
points = pd.merge(points, dd, on='UID')
points

,UID,lat,lon,Asset ID,Asset Name,Treatment,Block,District,State,Total_Expenditure,HSG,pH,OC,CEC,drainage_density
0,3734,24.391782,86.647062,34001162850,Dova nirman,1,deoghar,DEOGHAR,JHARKHAND,17869.0,13.0,6.2,0.58,15.0,0.007048
1,11503,22.683443,86.417664,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,1,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,3.0,7.9,0.75,43.0,0.006093
2,6424,23.960886,86.472827,34001109193,DOBHA CONT. OF BABUJAN SOREN,1,tundi,DHANBAD,JHARKHAND,0.0,3.0,7.6,1.40,0.0,0.006046
3,15984,24.066874,85.733054,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,1,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,3.0,6.2,0.58,15.0,0.006411
4,23614,22.599589,85.430967,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,1,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,13.0,4.9,0.99,8.0,0.006100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,93929,21.481009,85.008159,0,control,0,barkote,DEOGARH,ODISHA,0.0,3.0,6.3,0.53,6.0,0.004737
9996,80212,22.360100,85.155573,0,control,0,manoharpur,WEST SINGHBHUM,JHARKHAND,0.0,3.0,7.6,1.40,0.0,0.006034
9997,72004,18.537050,83.362625,0,control,0,Bobbili,VIZIANAGARAM,ANDHRA PRADESH,0.0,14.0,6.2,0.58,15.0,0.005739
9998,36536,24.419039,87.009157,0,control,0,jarmundi,DUMKA,JHARKHAND,0.0,3.0,6.2,0.58,15.0,0.005852


In [ ]:
points.columns

Index(['UID', 'lat', 'lon', 'Asset ID', 'Asset Name', 'Treatment', 'Block',
       'District', 'State', 'Total_Expenditure', 'HSG', 'pH', 'OC', 'CEC',
       'drainage_density'],
      dtype='object')

In [ ]:
new_df = pd.read_csv("covariates_sampled_points.csv")

In [ ]:
new_df.head()

,system:index,Asset ID,Asset Name,Block,District,State,Total_Expenditure,Treatment,UID,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_road,dist_closest_upstream_forest,elevation,flow_accumulation,lat,lon,proximity_water,recent_cropping_intensity,slope,var,.geo
0,0,34001162850,Dova nirman,deoghar,DEOGHAR,JHARKHAND,17869.0,1,3734,0.000000,2757.812358,2788.872635,2218.476681,39.261002,235.364174,66,24.391782,86.647062,2.398827,1.0,2.689599,[<Feature>],"{""type"":""Point"",""coordinates"":[86.647062,24.39..."
1,1,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,1,11503,103.653843,4283.513701,2289.282683,324.437972,0.000000,127.948709,1,22.683443,86.417664,5.554252,1.0,2.807031,[<Feature>],"{""type"":""Point"",""coordinates"":[86.417664,22.68..."
2,2,34001109193,DOBHA CONT. OF BABUJAN SOREN,tundi,DHANBAD,JHARKHAND,0.0,1,6424,0.000000,244.475233,3670.598791,334.427904,0.000000,242.235129,1,23.960886,86.472827,0.771261,1.0,2.403286,[<Feature>],"{""type"":""Point"",""coordinates"":[86.472827,23.96..."
3,3,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,1,15984,0.000000,1881.011919,1464.235109,124.153419,0.000000,465.175694,25,24.066874,85.733054,0.865103,2.0,4.108871,[<Feature>],"{""type"":""Point"",""coordinates"":[85.733054,24.06..."
4,4,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,1,23614,9.585659,0.000000,298.475078,194.134399,38.342744,295.016536,3,22.599589,85.430967,3.636364,1.0,2.925608,[<Feature>],"{""type"":""Point"",""coordinates"":[85.430967,22.59..."


In [ ]:
points = pd.merge(points, new_df, on='UID')

In [ ]:
# cols_list = ['UID', 'lat', 'lon', 'Treatment', 'dist_closest_crop', 'dist_closest_lin', 'dist_closest_river', 'dist_closest_upstream_forest', 'elevation', 'flow_accumulation', 'proximity_water', 'recent_cropping_intensity', 'slope', 'HSG', 'pH', 'OC', 'CEC', 'drainage_density', '.geo']
cols_list = ['UID', 'Asset ID', 'Asset Name', 'Block', 'District', 'State',
       'Total_Expenditure', 'system:index', 'Treatment', 'dist_closest_crop',
       'dist_closest_lin', 'dist_closest_river', 'dist_closest_road',
       'dist_closest_upstream_forest', 'elevation', 'flow_accumulation', 'lat',
       'lon', 'proximity_water', 'recent_cropping_intensity', 'slope', 'var',
       '.geo', 'HSG', 'pH', 'OC', 'CEC']

In [ ]:
points.head()

,UID,lat_x,lon_x,Asset ID_x,Asset Name_x,Treatment_x,Block_x,District_x,State_x,Total_Expenditure_x,HSG,pH,OC,CEC,drainage_density,system:index,Asset ID_y,Asset Name_y,Block_y,District_y,State_y,Total_Expenditure_y,Treatment_y,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_road,dist_closest_upstream_forest,elevation,flow_accumulation,lat_y,lon_y,proximity_water,recent_cropping_intensity,slope,var,.geo
0,3734,24.391782,86.647062,34001162850,Dova nirman,1,deoghar,DEOGHAR,JHARKHAND,17869.0,13.0,6.2,0.58,15.0,0.007048,0,34001162850,Dova nirman,deoghar,DEOGHAR,JHARKHAND,17869.0,1,0.000000,2757.812358,2788.872635,2218.476681,39.261002,235.364174,66,24.391782,86.647062,2.398827,1.0,2.689599,[<Feature>],"{""type"":""Point"",""coordinates"":[86.647062,24.39..."
1,11503,22.683443,86.417664,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,1,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,3.0,7.9,0.75,43.0,0.006093,1,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,1,103.653843,4283.513701,2289.282683,324.437972,0.000000,127.948709,1,22.683443,86.417664,5.554252,1.0,2.807031,[<Feature>],"{""type"":""Point"",""coordinates"":[86.417664,22.68..."
2,6424,23.960886,86.472827,34001109193,DOBHA CONT. OF BABUJAN SOREN,1,tundi,DHANBAD,JHARKHAND,0.0,3.0,7.6,1.40,0.0,0.006046,2,34001109193,DOBHA CONT. OF BABUJAN SOREN,tundi,DHANBAD,JHARKHAND,0.0,1,0.000000,244.475233,3670.598791,334.427904,0.000000,242.235129,1,23.960886,86.472827,0.771261,1.0,2.403286,[<Feature>],"{""type"":""Point"",""coordinates"":[86.472827,23.96..."
3,15984,24.066874,85.733054,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,1,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,3.0,6.2,0.58,15.0,0.006411,3,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,1,0.000000,1881.011919,1464.235109,124.153419,0.000000,465.175694,25,24.066874,85.733054,0.865103,2.0,4.108871,[<Feature>],"{""type"":""Point"",""coordinates"":[85.733054,24.06..."
4,23614,22.599589,85.430967,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,1,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,13.0,4.9,0.99,8.0,0.006100,4,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,1,9.585659,0.000000,298.475078,194.134399,38.342744,295.016536,3,22.599589,85.430967,3.636364,1.0,2.925608,[<Feature>],"{""type"":""Point"",""coordinates"":[85.430967,22.59..."


In [ ]:
# Drop all columns with the '_y' suffix if you want to retain only the '_x' versions
columns_to_drop = [col for col in points.columns if col.endswith('_y')]
points_cleaned = points.drop(columns=columns_to_drop)

# Optionally, rename columns to remove '_x' suffix
points_cleaned.columns = [col.replace('_x', '') for col in points_cleaned.columns]

# Display the cleaned DataFrame to verify changes
print(points_cleaned.head())

     UID        lat        lon     Asset ID  \
0   3734  24.391782  86.647062  34001162850   
1  11503  22.683443  86.417664  34000979364   
2   6424  23.960886  86.472827  34001109193   
3  15984  24.066874  85.733054  34000926893   
4  23614  22.599589  85.430967  34000828570   

                                          Asset Name  Treatment       Block  \
0                                        Dova nirman          1     deoghar   
1  राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...          1   ghatshila   
2                       DOBHA CONT. OF BABUJAN SOREN          1       tundi   
3  KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...          1  bishnugarh   
4  Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...          1       sonua   

         District      State  Total_Expenditure   HSG   pH    OC   CEC  \
0         DEOGHAR  JHARKHAND            17869.0  13.0  6.2  0.58  15.0   
1   EAST SINGHBUM  JHARKHAND                0.0   3.0  7.9  0.75  43.0   
2         DHANBAD  J

In [ ]:
# Reorder columns
points = points_cleaned
points = points[cols_list]
points = points.drop('var', axis = 1, errors='ignore')

✅Change AEZ

In [ ]:
# Output final all_covariates file
points.to_csv("AEZ_12_static_covariates_all.csv", index=False)
points

,UID,Asset ID,Asset Name,Block,District,State,Total_Expenditure,system:index,Treatment,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_road,dist_closest_upstream_forest,elevation,flow_accumulation,lat,lon,proximity_water,recent_cropping_intensity,slope,.geo,HSG,pH,OC,CEC
0,3734,34001162850,Dova nirman,deoghar,DEOGHAR,JHARKHAND,17869.0,0,1,0.000000,2757.812358,2788.872635,2218.476681,39.261002,235.364174,66,24.391782,86.647062,2.398827,1.0,2.689599,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",13.0,6.2,0.58,15.0
1,11503,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,1,1,103.653843,4283.513701,2289.282683,324.437972,0.000000,127.948709,1,22.683443,86.417664,5.554252,1.0,2.807031,"{""type"":""Point"",""coordinates"":[86.417664,22.68...",3.0,7.9,0.75,43.0
2,6424,34001109193,DOBHA CONT. OF BABUJAN SOREN,tundi,DHANBAD,JHARKHAND,0.0,2,1,0.000000,244.475233,3670.598791,334.427904,0.000000,242.235129,1,23.960886,86.472827,0.771261,1.0,2.403286,"{""type"":""Point"",""coordinates"":[86.472827,23.96...",3.0,7.6,1.40,0.0
3,15984,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,3,1,0.000000,1881.011919,1464.235109,124.153419,0.000000,465.175694,25,24.066874,85.733054,0.865103,2.0,4.108871,"{""type"":""Point"",""coordinates"":[85.733054,24.06...",3.0,6.2,0.58,15.0
4,23614,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,4,1,9.585659,0.000000,298.475078,194.134399,38.342744,295.016536,3,22.599589,85.430967,3.636364,1.0,2.925608,"{""type"":""Point"",""coordinates"":[85.430967,22.59...",13.0,4.9,0.99,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,93929,0,control,barkote,DEOGARH,ODISHA,0.0,2495,0,21.517102,808.310347,2934.491189,2238.340294,216.244789,128.140343,1,21.481009,85.008159,160.768328,2.0,2.290708,"{""type"":""Point"",""coordinates"":[85.008159,21.48...",3.0,6.3,0.53,6.0
9996,80212,0,control,manoharpur,WEST SINGHBHUM,JHARKHAND,0.0,2496,0,0.000000,1957.338562,1467.290659,153.745271,157.349845,226.292669,1,22.360100,85.155573,6.354839,1.0,2.003794,"{""type"":""Point"",""coordinates"":[85.155573,22.36...",3.0,7.6,1.40,0.0
9997,72004,0,control,Bobbili,VIZIANAGARAM,ANDHRA PRADESH,0.0,2497,0,9.711060,805.783805,1041.712526,482.197404,38.844331,109.937872,6,18.537050,83.362625,37.492669,1.0,2.848030,"{""type"":""Point"",""coordinates"":[83.362625,18.53...",14.0,6.2,0.58,15.0
9998,36536,0,control,jarmundi,DUMKA,JHARKHAND,0.0,2498,0,0.000000,2590.314052,5068.611161,1854.365307,127.028450,223.774791,1,24.419039,87.009157,0.076246,1.0,3.225507,"{""type"":""Point"",""coordinates"":[87.009157,24.41...",3.0,6.2,0.58,15.0


In [ ]:
# File which contains the objectids for eachh points. This was output in the watershed level precipitation script (Precipitation_WatershedLevel_KB.ipynb)
objid = pd.read_csv("AEZ12_all_points_with_ws_objectid.csv")
# objid = objid.drop_duplicates(subset='UID', keep='first')
objid

,system:index,Asset ID,Asset Name,Block,District,State,Total_Expenditure,Treatment,UID,objectid,pointInRegion,.geo
0,0000000000000000009a,2006232785,Farm Pond,Addateegala,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,68317.0,1,284,1417,NaN,"{""type"":""Point"",""coordinates"":[82.112366058338..."
1,00000000000000000149,2005944052,Farm Pond,Addateegala,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,263.0,1,281,1420,NaN,"{""type"":""Point"",""coordinates"":[82.070459426637..."
2,00000000000000000530,2006940969,Farm Pond,Addateegala,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,47333.0,1,292,1420,NaN,"{""type"":""Point"",""coordinates"":[82.031660788371..."
3,000000000000000007ef,2005375498,Farm Pond,Addateegala,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,83083.0,1,279,1420,NaN,"{""type"":""Point"",""coordinates"":[82.044489622102..."
4,00000000000000000097,2006445761,Farm Pond,Hukumpeta,ALLURI SITHARAMA RAJU,ANDHRA PRADESH,32524.0,1,76,349,NaN,"{""type"":""Point"",""coordinates"":[82.852964775001..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,00000000000000000494,34001166956,dova,jagannathpur,WEST SINGHBHUM,JHARKHAND,837.0,1,22845,1109,NaN,"{""type"":""Point"",""coordinates"":[85.602642685002..."
9996,00000000000000000510,34001011137,dova,jagannathpur,WEST SINGHBHUM,JHARKHAND,19488.0,1,22940,1109,NaN,"{""type"":""Point"",""coordinates"":[85.618227243184..."
9997,00000000000000000679,34000771696,dova,jagannathpur,WEST SINGHBHUM,JHARKHAND,17201.0,1,22955,1109,NaN,"{""type"":""Point"",""coordinates"":[85.665890574344..."
9998,000000000000000008f4,34001216221,dova,jagannathpur,WEST SINGHBHUM,JHARKHAND,20040.0,1,22882,1109,NaN,"{""type"":""Point"",""coordinates"":[85.673792101411..."


In [ ]:
# Keep columns that you need. Remove others
# objid = objid[['UID', 'Asset ID','objectid', 'Block', 'State', 'District']]
objid = objid[['UID','objectid']]
objid

,UID,objectid
0,284,1417
1,281,1420
2,292,1420
3,279,1420
4,76,349
...,...,...
9995,22845,1109
9996,22940,1109
9997,22955,1109
9998,22882,1109


In [ ]:
points = pd.merge(points, objid, on='UID')

In [ ]:
points

,UID,Asset ID,Asset Name,Block,District,State,Total_Expenditure,system:index,Treatment,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_road,dist_closest_upstream_forest,elevation,flow_accumulation,lat,lon,proximity_water,recent_cropping_intensity,slope,.geo,HSG,pH,OC,CEC,objectid
0,3734,34001162850,Dova nirman,deoghar,DEOGHAR,JHARKHAND,17869.0,0,1,0.000000,2757.812358,2788.872635,2218.476681,39.261002,235.364174,66,24.391782,86.647062,2.398827,1.0,2.689599,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",13.0,6.2,0.58,15.0,2005
1,11503,34000979364,राम पिन्द्राबाद मैं रविन्द्र नाथ महतो के जमीन ...,ghatshila,EAST SINGHBUM,JHARKHAND,0.0,1,1,103.653843,4283.513701,2289.282683,324.437972,0.000000,127.948709,1,22.683443,86.417664,5.554252,1.0,2.807031,"{""type"":""Point"",""coordinates"":[86.417664,22.68...",3.0,7.9,0.75,43.0,3171
2,6424,34001109193,DOBHA CONT. OF BABUJAN SOREN,tundi,DHANBAD,JHARKHAND,0.0,2,1,0.000000,244.475233,3670.598791,334.427904,0.000000,242.235129,1,23.960886,86.472827,0.771261,1.0,2.403286,"{""type"":""Point"",""coordinates"":[86.472827,23.96...",3.0,7.6,1.40,0.0,2064
3,15984,34000926893,KHARNA ME BHUNESHWAR MAHTO KE JAMIN ME DHOBHA ...,bishnugarh,HAZARIBAGH,JHARKHAND,42000.0,3,1,0.000000,1881.011919,1464.235109,124.153419,0.000000,465.175694,25,24.066874,85.733054,0.865103,2.0,4.108871,"{""type"":""Point"",""coordinates"":[85.733054,24.06...",3.0,6.2,0.58,15.0,2034
4,23614,34000828570,Gram Sarndiyaposh Main Vivek Kumar Katihaar Ke...,sonua,WEST SINGHBHUM,JHARKHAND,7560.0,4,1,9.585659,0.000000,298.475078,194.134399,38.342744,295.016536,3,22.599589,85.430967,3.636364,1.0,2.925608,"{""type"":""Point"",""coordinates"":[85.430967,22.59...",13.0,4.9,0.99,8.0,3172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,93929,0,control,barkote,DEOGARH,ODISHA,0.0,2495,0,21.517102,808.310347,2934.491189,2238.340294,216.244789,128.140343,1,21.481009,85.008159,160.768328,2.0,2.290708,"{""type"":""Point"",""coordinates"":[85.008159,21.48...",3.0,6.3,0.53,6.0,1161
9996,80212,0,control,manoharpur,WEST SINGHBHUM,JHARKHAND,0.0,2496,0,0.000000,1957.338562,1467.290659,153.745271,157.349845,226.292669,1,22.360100,85.155573,6.354839,1.0,2.003794,"{""type"":""Point"",""coordinates"":[85.155573,22.36...",3.0,7.6,1.40,0.0,1149
9997,72004,0,control,Bobbili,VIZIANAGARAM,ANDHRA PRADESH,0.0,2497,0,9.711060,805.783805,1041.712526,482.197404,38.844331,109.937872,6,18.537050,83.362625,37.492669,1.0,2.848030,"{""type"":""Point"",""coordinates"":[83.362625,18.53...",14.0,6.2,0.58,15.0,1364
9998,36536,0,control,jarmundi,DUMKA,JHARKHAND,0.0,2498,0,0.000000,2590.314052,5068.611161,1854.365307,127.028450,223.774791,1,24.419039,87.009157,0.076246,1.0,3.225507,"{""type"":""Point"",""coordinates"":[87.009157,24.41...",3.0,6.2,0.58,15.0,2480


In [ ]:
cols_list = ['UID', 'lat', 'lon', 'Treatment', 'Asset ID', 'objectid', 'Block', 'State', 'District', 'dist_closest_crop', 'dist_closest_lin', 'dist_closest_river', 'dist_closest_upstream_forest', 'elevation', 'flow_accumulation', 'proximity_water', 'recent_cropping_intensity', 'slope', 'HSG', 'pH', 'OC', 'CEC', '.geo']

In [ ]:
points = points.drop('system:index', axis=1, errors='ignore') # Extra column that is added by GEE
points = points[cols_list]
points

,UID,lat,lon,Treatment,Asset ID,objectid,Block,State,District,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,recent_cropping_intensity,slope,HSG,pH,OC,CEC,.geo
0,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,1.0,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39..."
1,11503,22.683443,86.417664,1,34000979364,3171,ghatshila,JHARKHAND,EAST SINGHBUM,103.653843,4283.513701,2289.282683,0.000000,127.948709,1,5.554252,1.0,2.807031,3.0,7.9,0.75,43.0,"{""type"":""Point"",""coordinates"":[86.417664,22.68..."
2,6424,23.960886,86.472827,1,34001109193,2064,tundi,JHARKHAND,DHANBAD,0.000000,244.475233,3670.598791,0.000000,242.235129,1,0.771261,1.0,2.403286,3.0,7.6,1.40,0.0,"{""type"":""Point"",""coordinates"":[86.472827,23.96..."
3,15984,24.066874,85.733054,1,34000926893,2034,bishnugarh,JHARKHAND,HAZARIBAGH,0.000000,1881.011919,1464.235109,0.000000,465.175694,25,0.865103,2.0,4.108871,3.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[85.733054,24.06..."
4,23614,22.599589,85.430967,1,34000828570,3172,sonua,JHARKHAND,WEST SINGHBHUM,9.585659,0.000000,298.475078,38.342744,295.016536,3,3.636364,1.0,2.925608,13.0,4.9,0.99,8.0,"{""type"":""Point"",""coordinates"":[85.430967,22.59..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,93929,21.481009,85.008159,0,0,1161,barkote,ODISHA,DEOGARH,21.517102,808.310347,2934.491189,216.244789,128.140343,1,160.768328,2.0,2.290708,3.0,6.3,0.53,6.0,"{""type"":""Point"",""coordinates"":[85.008159,21.48..."
9996,80212,22.360100,85.155573,0,0,1149,manoharpur,JHARKHAND,WEST SINGHBHUM,0.000000,1957.338562,1467.290659,157.349845,226.292669,1,6.354839,1.0,2.003794,3.0,7.6,1.40,0.0,"{""type"":""Point"",""coordinates"":[85.155573,22.36..."
9997,72004,18.537050,83.362625,0,0,1364,Bobbili,ANDHRA PRADESH,VIZIANAGARAM,9.711060,805.783805,1041.712526,38.844331,109.937872,6,37.492669,1.0,2.848030,14.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[83.362625,18.53..."
9998,36536,24.419039,87.009157,0,0,2480,jarmundi,JHARKHAND,DUMKA,0.000000,2590.314052,5068.611161,127.028450,223.774791,1,0.076246,1.0,3.225507,3.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[87.009157,24.41..."


✅Change AEZ

In [ ]:
points.to_csv("AEZ_12_all_covariates_final.csv", index=False)

#Step 6 - Create DF for all seasons

Now we construct of the final DF which is used for the analysis. It inputs the above mentioned files (with the exception of drought indicators for RQ1).
✅Please run this separately for all 3 seasons


In [ ]:
# Set these configuration variables according to need:
# Setting 1: Remove drought years

today = date.today()
str_date = today.strftime("%b-%d-%Y")

# str_date = "Jan-22-2024" # if want to use old dumped df
season = 3 # 1 (Kharif), 2 (Rabi), 3 (Zaid)
# freq_threshold = 5 # only for setting 1
dataset = 1 # 1 (built in 2017), 2 (valid farm ponds)
psm_matched = 0 # 0/1 if you want psm matched dataset. If 0, then #cp = 3*#tp

✅Change Season

In [ ]:
if season == 1:
    season_name = "Kharif"
elif season == 2:
    season_name = "Rabi"
elif season == 3:
    season_name = "Zaid"
else:
    raise Exception("Please set season value")
season_name

'Zaid'

✅Change AEZ

In [ ]:
csvfile = 'AEZ_12_all_covariates_final.csv'
df = pd.read_csv(csvfile)
df = df.drop(['Unnamed: 0'], axis=1, errors='ignore')

In [ ]:
retain_cols = [col for col in df.columns if 'frequency' not in col and 'intensity' not in col and 'drought' not in col] +['recent_cropping_intensity']
new_cols = ['base_year', 'target_year']
df_base_target = pd.DataFrame(columns = retain_cols + new_cols)
df_base_target

,UID,lat,lon,Treatment,Asset ID,objectid,Block,State,District,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,.geo,recent_cropping_intensity,base_year,target_year


In [ ]:
# 2B:   takes approx 8 minutes
for index, row in df.iterrows():
  dict = {key: row[key] for key in retain_cols}
  for target_year in range(2018, 2022):
    dict['base_year'] = str(2016)
    dict['target_year'] = str(target_year)
    df_append = pd.DataFrame([dict])
    df_base_target = pd.concat([df_base_target, df_append], ignore_index=True)

<ipython-input-276-743c25bedf85>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_base_target = pd.concat([df_base_target, df_append], ignore_index=True)


In [ ]:
df_base_target.shape

(40000, 25)

In [ ]:
df_base_target.to_csv(f'Allyears_df_base_target_{season_name}_{str_date}.csv', index=False)

In [ ]:
df_final = pd.read_csv(f'Allyears_df_base_target_{season_name}_{str_date}.csv')
df_final.shape

(40000, 25)

✅Change AEZ

In [ ]:
fc = ee.FeatureCollection('users/mtpictd/NDVI/mean_of_mean_unionmask_AEZ12_2016-2023_Landsat_v2')

# Get a download URL for the FeatureCollection.
download_url = fc.getDownloadURL(**{
  'filetype': 'csv',
})
print('URL for downloading FeatureCollection as CSV:', download_url)

URL for downloading FeatureCollection as CSV: https://earthengine.googleapis.com/v1/projects/ee-mtpictd/tables/a458f7f445fb9b7f81e51b700c51a919-68039445dafd10e1fa5799dd927ed97e:getFeatures


In [ ]:
%pwd

'/content/drive/MyDrive/Sanya/DID-AEZ-12'

✅Change AEZ

In [ ]:
original_cols = [col for col in df_final.columns]
df_ndvi = pd.read_csv('AEZ12_NDVI_unionmask_all_points.csv')
print(df_ndvi)
get_cols = []
all_cols = list(df_ndvi.columns)
for col in all_cols:
  if 'ndvi' in col or 'ndmi' in col or 'GCI' in col:
    get_cols.append(col)
get_cols.append('UID')
df_ndvi = df_ndvi[get_cols]
df_ndvi.shape

              system:index     Asset ID Asset Name        Block  \
0     00000000000000000023            0    control  Addateegala   
1     00000000000000000024            0    control  Addateegala   
2     00000000000000000025            0    control  Addateegala   
3     00000000000000000026            0    control  Addateegala   
4     00000000000000000027            0    control  Addateegala   
...                    ...          ...        ...          ...   
9995  000000000000000026f0  34000911415      dobha     goelkera   
9996  000000000000000026ff  34001026109      dobha     noamundi   
9997  00000000000000002700  34001219035      dobha     noamundi   
9998  00000000000000002702  34001517659      dobha     noamundi   
9999  00000000000000002703  34001474295      dobha        tonto   

                   District  GCI_Kharif_2016  GCI_Kharif_2017  \
0     ALLURI SITHARAMA RAJU         0.776679         0.526054   
1     ALLURI SITHARAMA RAJU         0.605267         0.607066   


(10000, 64)

In [ ]:
df_main = pd.merge(df_final, df_ndvi, on='UID')
df_main.shape
df_main.head()

,UID,lat,lon,Treatment,Asset ID,objectid,Block,State,District,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,.geo,recent_cropping_intensity,base_year,target_year,GCI_Kharif_2016,GCI_Kharif_2017,GCI_Kharif_2018,GCI_Kharif_2019,GCI_Kharif_2020,GCI_Kharif_2021,GCI_Kharif_2022,GCI_Rabi_2016,GCI_Rabi_2017,GCI_Rabi_2018,GCI_Rabi_2019,GCI_Rabi_2020,GCI_Rabi_2021,GCI_Rabi_2022,GCI_Zaid_2016,GCI_Zaid_2017,GCI_Zaid_2018,GCI_Zaid_2019,GCI_Zaid_2020,GCI_Zaid_2021,GCI_Zaid_2022,ndmi_Kharif_2016,ndmi_Kharif_2017,ndmi_Kharif_2018,ndmi_Kharif_2019,ndmi_Kharif_2020,ndmi_Kharif_2021,ndmi_Kharif_2022,ndmi_Rabi_2016,ndmi_Rabi_2017,ndmi_Rabi_2018,ndmi_Rabi_2019,ndmi_Rabi_2020,ndmi_Rabi_2021,ndmi_Rabi_2022,ndmi_Zaid_2016,ndmi_Zaid_2017,ndmi_Zaid_2018,ndmi_Zaid_2019,ndmi_Zaid_2020,ndmi_Zaid_2021,ndmi_Zaid_2022,ndvi_Kharif_2016,ndvi_Kharif_2017,ndvi_Kharif_2018,ndvi_Kharif_2019,ndvi_Kharif_2020,ndvi_Kharif_2021,ndvi_Kharif_2022,ndvi_Rabi_2016,ndvi_Rabi_2017,ndvi_Rabi_2018,ndvi_Rabi_2019,ndvi_Rabi_2020,ndvi_Rabi_2021,ndvi_Rabi_2022,ndvi_Zaid_2016,ndvi_Zaid_2017,ndvi_Zaid_2018,ndvi_Zaid_2019,ndvi_Zaid_2020,ndvi_Zaid_2021,ndvi_Zaid_2022
0,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2018,0.741432,0.612028,0.312804,0.430948,0.335405,0.783166,0.749935,0.536896,0.569893,0.464871,0.586973,0.488551,0.619931,0.613395,0.376939,0.442472,0.342904,0.511011,0.454736,0.587723,0.505015,0.087753,0.167762,0.140699,0.212999,0.118943,0.120036,0.138541,-0.047121,-0.006949,-0.043874,0.014933,-0.074427,-0.001993,-0.052231,-0.020645,-0.027907,0.000055,0.029239,-0.038038,-0.024894,-0.061846,0.263662,0.216719,0.112903,0.156309,0.137063,0.288058,0.270127,0.168133,0.187633,0.14544,0.195445,0.157208,0.209492,0.19753,0.113508,0.138468,0.109816,0.184419,0.140301,0.20451,0.165923
1,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2019,0.741432,0.612028,0.312804,0.430948,0.335405,0.783166,0.749935,0.536896,0.569893,0.464871,0.586973,0.488551,0.619931,0.613395,0.376939,0.442472,0.342904,0.511011,0.454736,0.587723,0.505015,0.087753,0.167762,0.140699,0.212999,0.118943,0.120036,0.138541,-0.047121,-0.006949,-0.043874,0.014933,-0.074427,-0.001993,-0.052231,-0.020645,-0.027907,0.000055,0.029239,-0.038038,-0.024894,-0.061846,0.263662,0.216719,0.112903,0.156309,0.137063,0.288058,0.270127,0.168133,0.187633,0.14544,0.195445,0.157208,0.209492,0.19753,0.113508,0.138468,0.109816,0.184419,0.140301,0.20451,0.165923
2,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2020,0.741432,0.612028,0.312804,0.430948,0.335405,0.783166,0.749935,0.536896,0.569893,0.464871,0.586973,0.488551,0.619931,0.613395,0.376939,0.442472,0.342904,0.511011,0.454736,0.587723,0.505015,0.087753,0.167762,0.140699,0.212999,0.118943,0.120036,0.138541,-0.047121,-0.006949,-0.043874,0.014933,-0.074427,-0.001993,-0.052231,-0.020645,-0.027907,0.000055,0.029239,-0.038038,-0.024894,-0.061846,0.263662,0.216719,0.112903,0.156309,0.137063,0.288058,0.270127,0.168133,0.187633,0.14544,0.195445,0.157208,0.209492,0.19753,0.113508,0.138468,0.109816,0.184419,0.140301,0.20451,0.165923
3,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2021,0.741432,0.612028,0.312804,0.430948,0.335405,0.783166,0.749935,0.536896,0.569893,0.464871,0.586

In [ ]:
nan_counts = df_ndvi.isna().sum()
print(nan_counts)

GCI_Kharif_2016    278
GCI_Kharif_2017    278
GCI_Kharif_2018    278
GCI_Kharif_2019    278
GCI_Kharif_2020    278
                  ... 
ndvi_Zaid_2019     278
ndvi_Zaid_2020     278
ndvi_Zaid_2021     278
ndvi_Zaid_2022     278
UID                  0
Length: 64, dtype: int64


In [ ]:
df_main_global = df_main.copy()

In [ ]:
for index, row in df_main.iterrows():
  df_main.loc[index, f'ndvi_{season_name}_max'] = max(row[f'ndvi_{season_name}_2016'], row[f'ndvi_{season_name}_2017'], row[f'ndvi_{season_name}_2018'], row[f'ndvi_{season_name}_2019'], row[f'ndvi_{season_name}_2020'], row[f'ndvi_{season_name}_2021'], row[f'ndvi_{season_name}_2022'])
  df_main.loc[index, f'ndvi_{season_name}_min'] = min(row[f'ndvi_{season_name}_2016'], row[f'ndvi_{season_name}_2017'], row[f'ndvi_{season_name}_2018'], row[f'ndvi_{season_name}_2019'], row[f'ndvi_{season_name}_2020'], row[f'ndvi_{season_name}_2021'], row[f'ndvi_{season_name}_2022'])
for index, row in df_main.iterrows():
  df_main.loc[index, f'ndmi_{season_name}_max'] = max(row[f'ndmi_{season_name}_2016'], row[f'ndmi_{season_name}_2017'], row[f'ndmi_{season_name}_2018'], row[f'ndmi_{season_name}_2019'], row[f'ndmi_{season_name}_2020'], row[f'ndmi_{season_name}_2021'], row[f'ndmi_{season_name}_2022'])
  df_main.loc[index, f'ndmi_{season_name}_min'] = min(row[f'ndmi_{season_name}_2016'], row[f'ndmi_{season_name}_2017'], row[f'ndmi_{season_name}_2018'], row[f'ndmi_{season_name}_2019'], row[f'ndmi_{season_name}_2020'], row[f'ndmi_{season_name}_2021'], row[f'ndmi_{season_name}_2022'])
for index, row in df_main.iterrows():
  df_main.loc[index, f'GCI_{season_name}_max'] = max(row[f'GCI_{season_name}_2016'], row[f'GCI_{season_name}_2017'], row[f'GCI_{season_name}_2018'], row[f'GCI_{season_name}_2019'], row[f'GCI_{season_name}_2020'], row[f'GCI_{season_name}_2021'], row[f'GCI_{season_name}_2022'])
  df_main.loc[index, f'GCI_{season_name}_min'] = min(row[f'GCI_{season_name}_2016'], row[f'GCI_{season_name}_2017'], row[f'GCI_{season_name}_2018'], row[f'GCI_{season_name}_2019'], row[f'GCI_{season_name}_2020'], row[f'GCI_{season_name}_2021'], row[f'GCI_{season_name}_2022'])

In [ ]:
new_cols1 = [f'ndvi_{season_name}_target', f'ndvi_{season_name}_target_norm', f'ndvi_{season_name}_base', f'ndvi_{season_name}_base_norm']
new_cols2 = [f'ndmi_{season_name}_target', f'ndmi_{season_name}_target_norm', f'ndmi_{season_name}_base', f'ndmi_{season_name}_base_norm']
new_cols2 = [f'GCI_{season_name}_target', f'GCI_{season_name}_target_norm', f'GCI_{season_name}_base', f'GCI_{season_name}_base_norm']
df_main_ndvi = pd.DataFrame(columns = original_cols + new_cols1 + new_cols2)
df_main_ndvi.head()

,UID,lat,lon,Treatment,Asset ID,objectid,Block,State,District,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,.geo,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm


In [ ]:
# takes time. Approx 10 mins for 40k rows
for index, row in df_main.iterrows():
  dict = {key: row[key] for key in original_cols}
  base_year = row['base_year']
  target_year = row['target_year']
  dict[f'ndvi_{season_name}_base'] = row[f'ndvi_{season_name}_{base_year}']
  dict[f'ndvi_{season_name}_base_norm'] = (row[f'ndvi_{season_name}_{base_year}'] - row[f'ndvi_{season_name}_min'])/(row[f'ndvi_{season_name}_max'] - row[f'ndvi_{season_name}_min'] + 0.00001)
  dict[f'ndvi_{season_name}_target'] = row[f'ndvi_{season_name}_{target_year}']
  dict[f'ndvi_{season_name}_target_norm'] = (row[f'ndvi_{season_name}_{target_year}'] - row[f'ndvi_{season_name}_min'])/(row[f'ndvi_{season_name}_max'] - row[f'ndvi_{season_name}_min'] + 0.00001)

  dict[f'ndmi_{season_name}_base'] = row[f'ndmi_{season_name}_{base_year}']
  dict[f'ndmi_{season_name}_base_norm'] = (row[f'ndmi_{season_name}_{base_year}'] - row[f'ndmi_{season_name}_min'])/(row[f'ndmi_{season_name}_max'] - row[f'ndmi_{season_name}_min'] + 0.00001)
  dict[f'ndmi_{season_name}_target'] = row[f'ndmi_{season_name}_{target_year}']
  dict[f'ndmi_{season_name}_target_norm'] = (row[f'ndmi_{season_name}_{target_year}'] - row[f'ndmi_{season_name}_min'])/(row[f'ndmi_{season_name}_max'] - row[f'ndmi_{season_name}_min'] + 0.00001)

  dict[f'GCI_{season_name}_base'] = row[f'GCI_{season_name}_{base_year}']
  dict[f'GCI_{season_name}_base_norm'] = (row[f'GCI_{season_name}_{base_year}'] - row[f'GCI_{season_name}_min'])/(row[f'GCI_{season_name}_max'] - row[f'GCI_{season_name}_min'] + 0.00001)
  dict[f'GCI_{season_name}_target'] = row[f'GCI_{season_name}_{target_year}']
  dict[f'GCI_{season_name}_target_norm'] = (row[f'GCI_{season_name}_{target_year}'] - row[f'GCI_{season_name}_min'])/(row[f'GCI_{season_name}_max'] - row[f'GCI_{season_name}_min'] + 0.00001)
  df_append = pd.DataFrame([dict])
  df_main_ndvi = pd.concat([df_main_ndvi, df_append], ignore_index=True)
  # df_non_drought_new = df_non_drought_new.append(dict, ignore_index=True)

<ipython-input-288-c388ae28d8df>:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_main_ndvi = pd.concat([df_main_ndvi, df_append], ignore_index=True)


In [ ]:
df_main_ndvi[f'{season_name}_ndvi_diff'] = df_main_ndvi[f'ndvi_{season_name}_target'] - df_main_ndvi[f'ndvi_{season_name}_base']
df_main_ndvi[f'{season_name}_ndvi_diff_norm'] = df_main_ndvi[f'ndvi_{season_name}_target_norm'] - df_main_ndvi[f'ndvi_{season_name}_base_norm']

df_main_ndvi[f'{season_name}_ndmi_diff'] = df_main_ndvi[f'ndmi_{season_name}_target'] - df_main_ndvi[f'ndmi_{season_name}_base']
df_main_ndvi[f'{season_name}_ndmi_diff_norm'] = df_main_ndvi[f'ndmi_{season_name}_target_norm'] - df_main_ndvi[f'ndmi_{season_name}_base_norm']

df_main_ndvi[f'{season_name}_GCI_diff'] = df_main_ndvi[f'GCI_{season_name}_target'] - df_main_ndvi[f'GCI_{season_name}_base']
df_main_ndvi[f'{season_name}_GCI_diff_norm'] = df_main_ndvi[f'GCI_{season_name}_target_norm'] - df_main_ndvi[f'GCI_{season_name}_base_norm']

In [ ]:
df_main_ndvi.head()
df_main_ndvi.shape

(40000, 43)

In [ ]:
df_main_ndvi.to_csv(f'Allyears_dump_df_main_with_ndvi_ndmi_gci_{season_name}_{str_date}.csv', index=False)

In [ ]:
df_ndvicalc = pd.read_csv(f'Allyears_dump_df_main_with_ndvi_ndmi_gci_{season_name}_{str_date}.csv')
df_ndvicalc.shape

(40000, 43)

In [ ]:
df_ndvicalc

,UID,lat,lon,Treatment,Asset ID,objectid,Block,State,District,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,.geo,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm,ndmi_Zaid_base,ndmi_Zaid_base_norm,ndmi_Zaid_target,ndmi_Zaid_target_norm,Zaid_ndvi_diff,Zaid_ndvi_diff_norm,Zaid_ndmi_diff,Zaid_ndmi_diff_norm,Zaid_GCI_diff,Zaid_GCI_diff_norm
0,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2018,0.109816,0.000000,0.113508,0.038989,0.342904,0.000000,0.376939,0.139014,-0.020645,0.452287,0.000055,0.679525,-0.003692,-0.038989,0.020700,0.227238,-0.034035,-0.139014
1,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2019,0.184419,0.787747,0.113508,0.038989,0.511011,0.686629,0.376939,0.139014,-0.020645,0.452287,0.029239,0.999890,0.070911,0.748758,0.049884,0.547604,0.134072,0.547615
2,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2020,0.140301,0.321897,0.113508,0.038989,0.454736,0.456775,0.376939,0.139014,-0.020645,0.452287,-0.038038,0.261357,0.026793,0.282908,-0.017393,-0.190930,0.077797,0.317761
3,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2021,0.204510,0.999894,0.113508,0.038989,0.587723,0.999959,0.376939,0.139014,-0.020645,0.452287,-0.024894,0.405645,0.091002,0.960906,-0.004249,-0.046641,0.210784,0.860945
4,11503,22.683443,86.417664,1,34000979364,3171,ghatshila,JHARKHAND,EAST SINGHBUM,103.653843,4283.513701,2289.282683,0.000000,127.948709,1,5.554252,2.807031,3.0,7.9,0.75,43.0,"{""type"":""Point"",""coordinates"":[86.417664,22.68...",1.0,2016,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,36536,24.419039,87.009157,0,0,2480,jarmundi,JHARKHAND,DUMKA,0.000000,2590.314052,5068.611161,127.028450,223.774791,1,0.076246,3.225507,3.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[87.009157,24.41...",1.0,2016,2021,0.157642,0.465349,0.122261,0.000000,0.420005,0.182883,0.390376,0.000000,-0.005018,0.389170,0.036539,0.999853,0.035380,0.465349,0.041556,0.610683,0.029629,0.182883
39996,69118,22.607766,86.247565,0,0,3170,potka,JHARKHAND,EAST SINGHBUM,0.000000,1432.621598,11.885868,48.876182,141.092525,2547,17.214076,2.564079,13.0,7.9,0.75,43.0,"{""type"":""Point"",""coordinates"":[86.247565,22.60...",1.0,2016,2018,0.179269,0.531284,0.135662,0.000000,0.478353,0.334933,0.422827,0.000000,-0.063889,0.000000,0.015779,0.999874,0.043607,0.531284,0.079668,0.999874,0.055526,0.334933
39997,69118,22.607766,86.247565,0,0,3170,potka,JHARKHAND,EAST SINGHBUM,0.000000,1432.621598,11.885868,48.876182,141.092525,2547,17.214076,2.564079,13.0,7.9,0.75,43.0,"{""type"":""Point"",""coordinates"":[86.247565,22.60...",1.0,2016,2019,0.217730,0.999878,0.135662,0.000000,0.588600,0.999940,0.422827,0.000000,-0.063889,0.000000,-0.002615,0.769015,0.082068,0.999878,0.061274,0.769015,0.165773,0.999940
39998,69118,22.607766,86.247565,0,0,3170,potka,JHARKHAND,EAST SINGHBUM,0.000000,1432.621598,11.885868,48.876182,

In [ ]:
df_vf = df_ndvicalc[df_ndvicalc["Treatment"] == 1]
df_cp = df_ndvicalc[df_ndvicalc["Treatment"] == 0]

In [ ]:
df = df_ndvicalc.copy()

✅Change AEZ

In [ ]:
df_prec_cp = pd.read_csv('AEZ12_Precipitation_data_watershed_lvl_global.csv').drop('Unnamed: 0', axis=1, errors='ignore')
df_prec_cp_new = df_prec_cp[['objectid']]

# can be coded better, but this is easy and it works
if season_name == "Kharif":
    for year in range(2015, 2022):
        df_prec_cp_new[f'precipitation_Kharif_{year}'] = df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(7)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(8)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(9)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(10)]
elif season_name == "Rabi":
    for year in range(2015, 2022):
        df_prec_cp_new[f'precipitation_Kharif_{year}'] = df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(7)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(8)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(9)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(10)]
        df_prec_cp_new['precipitation_Rabi_' + str(year)] = df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(11)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(12)] + df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(1)] + df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(2)]
elif season_name == "Zaid":
    for year in range(2015, 2022):
        df_prec_cp_new[f'precipitation_Kharif_{year}'] = df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(7)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(8)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(9)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(10)]
        df_prec_cp_new['precipitation_Rabi_' + str(year)] = df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(11)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(12)] + df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(1)] + df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(2)]
        df_prec_cp_new['precipitation_Zaid_' + str(year)] = df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(3)] + df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(4)] + df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(5)] + df_prec_cp['Precipitation' + '-' + str(year+1) + '-' + str(6)]

df = pd.merge(df, df_prec_cp_new, on='objectid', how='left')
# df = df.drop(['objectid'], axis=1)

<ipython-input-296-37b4d8b5eca0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prec_cp_new[f'precipitation_Kharif_{year}'] = df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(7)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(8)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(9)] + df_prec_cp['Precipitation' + '-' + str(year) + '-' + str(10)]
<ipython-input-296-37b4d8b5eca0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prec_cp_new['precipitation_Rabi_' + str(yea

In [ ]:
df_prec = df.copy()
df_prec.head()

,UID,lat,lon,Treatment,Asset ID,objectid,Block,State,District,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,.geo,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm,ndmi_Zaid_base,ndmi_Zaid_base_norm,ndmi_Zaid_target,ndmi_Zaid_target_norm,Zaid_ndvi_diff,Zaid_ndvi_diff_norm,Zaid_ndmi_diff,Zaid_ndmi_diff_norm,Zaid_GCI_diff,Zaid_GCI_diff_norm,precipitation_Kharif_2015,precipitation_Rabi_2015,precipitation_Zaid_2015,precipitation_Kharif_2016,precipitation_Rabi_2016,precipitation_Zaid_2016,precipitation_Kharif_2017,precipitation_Rabi_2017,precipitation_Zaid_2017,precipitation_Kharif_2018,precipitation_Rabi_2018,precipitation_Zaid_2018,precipitation_Kharif_2019,precipitation_Rabi_2019,precipitation_Zaid_2019,precipitation_Kharif_2020,precipitation_Rabi_2020,precipitation_Zaid_2020,precipitation_Kharif_2021,precipitation_Rabi_2021,precipitation_Zaid_2021
0,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2018,0.109816,0.000000,0.113508,0.038989,0.342904,0.000000,0.376939,0.139014,-0.020645,0.452287,0.000055,0.679525,-0.003692,-0.038989,0.020700,0.227238,-0.034035,-0.139014,568.475621,9.343147,262.636652,525.392053,7.632749,413.549218,853.67401,24.672765,311.072683,385.226648,102.324426,373.426855,734.113161,87.216267,398.951436,898.341936,8.973775,355.412280,624.465913,112.464428,202.874512
1,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2019,0.184419,0.787747,0.113508,0.038989,0.511011,0.686629,0.376939,0.139014,-0.020645,0.452287,0.029239,0.999890,0.070911,0.748758,0.049884,0.547604,0.134072,0.547615,568.475621,9.343147,262.636652,525.392053,7.632749,413.549218,853.67401,24.672765,311.072683,385.226648,102.324426,373.426855,734.113161,87.216267,398.951436,898.341936,8.973775,355.412280,624.465913,112.464428,202.874512
2,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2020,0.140301,0.321897,0.113508,0.038989,0.454736,0.456775,0.376939,0.139014,-0.020645,0.452287,-0.038038,0.261357,0.026793,0.282908,-0.017393,-0.190930,0.077797,0.317761,568.475621,9.343147,262.636652,525.392053,7.632749,413.549218,853.67401,24.672765,311.072683,385.226648,102.324426,373.426855,734.113161,87.216267,398.951436,898.341936,8.973775,355.412280,624.465913,112.464428,202.874512
3,3734,24.391782,86.647062,1,34001162850,2005,deoghar,JHARKHAND,DEOGHAR,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2021,0.204510,0.999894,0.113508,0.038989,0.587723,0.999959,0.376939,0.139014,-0.020645,0.452287,-0.024894,0.405645,0.091002,0.960906,-0.004249,-0.046641,0.210784,0.860945,568.475621,9.343147,262.636652,525.392053,7.632749,413.549218,853.67401,24.672765,311.072683,385.226648,102.324426,373.426855,734.113161,87.216267,398.951436,898.341936,8.973775,355.412280,624.465913,112.464428,202.874512
4,11503,22.683443,86.417664,1,34000979364,3171,ghatshila,JHARKHAND,EAST SINGHBUM,103.653843,4283.513701,2289.282683,0.000000,127.948709,1,5.554252,2.807031,3.0,7.9,0.75,43.0,"{""type"":""Point"",""coordinates"":[86.417664,22.68...",1.0,2016,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,753.080626,10.807642,442.439354,644.703565,4.589364,476.438801,1062.24355,33.78

In [ ]:
df_prec[f'precip_baseyr_{season_name}'] = 0
df_prec[f'precip_targetyr_{season_name}'] = 0
for index, row in df_prec.iterrows():
    base_year = row['base_year']
    target_year = row['target_year']
    df_prec.at[index, f'precip_baseyr_{season_name}'] = row[f'precipitation_{season_name}_{base_year}']
    df_prec.at[index, f'precip_targetyr_{season_name}'] = row[f'precipitation_{season_name}_{target_year}']
    if season_name == "Rabi" or season_name == "Zaid":
        df_prec.at[index, f'precip_baseyr_Kharif'] = row[f'precipitation_Kharif_{base_year}']
        df_prec.at[index, f'precip_targetyr_Kharif'] = row[f'precipitation_Kharif_{target_year}']
    if season_name == "Zaid":
        df_prec.at[index, f'precip_baseyr_Rabi'] = row[f'precipitation_Rabi_{base_year}']
        df_prec.at[index, f'precip_targetyr_Rabi'] = row[f'precipitation_Rabi_{target_year}']
# df_prec = df_prec.drop([f'precipitation_{season_name}_2015',f'precipitation_{season_name}_2016',f'precipitation_{season_name}_2017',f'precipitation_{season_name}_2018',f'precipitation_{season_name}_2019',f'precipitation_{season_name}_2020',f'precipitation_{season_name}_2021',f'precipitation_{season_name}_2022',], axis=1)
seasons = ["Kharif", "Rabi", "Zaid"]
for season in seasons:
    df_prec = df_prec.drop([f'precipitation_{season}_2015',f'precipitation_{season}_2016',f'precipitation_{season}_2017',f'precipitation_{season}_2018',f'precipitation_{season}_2019',f'precipitation_{season}_2020',f'precipitation_{season}_2021',], axis=1, errors='ignore')
df_prec.shape

<ipython-input-298-27e7a3ee300b>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '413.54921841331236' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_prec.at[index, f'precip_baseyr_{season_name}'] = row[f'precipitation_{season_name}_{base_year}']
<ipython-input-298-27e7a3ee300b>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '373.4268545945205' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_prec.at[index, f'precip_targetyr_{season_name}'] = row[f'precipitation_{season_name}_{target_year}']


(40000, 49)

In [ ]:
df_prec.to_csv(f"Allyears_NDVI_df_{season_name}_{str_date}.csv", index=False)

In [ ]:
df_main = pd.read_csv(f"Allyears_NDVI_df_{season_name}_{str_date}.csv")
# df_main = pd.read_csv("Allyears_NDVI_df_Zaid_Jan-17-2024.csv")

print((df_main.columns))

Index(['UID', 'lat', 'lon', 'Treatment', 'Asset ID', 'objectid', 'Block',
       'State', 'District', 'dist_closest_crop', 'dist_closest_lin',
       'dist_closest_river', 'dist_closest_upstream_forest', 'elevation',
       'flow_accumulation', 'proximity_water', 'slope', 'HSG', 'pH', 'OC',
       'CEC', '.geo', 'recent_cropping_intensity', 'base_year', 'target_year',
       'ndvi_Zaid_target', 'ndvi_Zaid_target_norm', 'ndvi_Zaid_base',
       'ndvi_Zaid_base_norm', 'GCI_Zaid_target', 'GCI_Zaid_target_norm',
       'GCI_Zaid_base', 'GCI_Zaid_base_norm', 'ndmi_Zaid_base',
       'ndmi_Zaid_base_norm', 'ndmi_Zaid_target', 'ndmi_Zaid_target_norm',
       'Zaid_ndvi_diff', 'Zaid_ndvi_diff_norm', 'Zaid_ndmi_diff',
       'Zaid_ndmi_diff_norm', 'Zaid_GCI_diff', 'Zaid_GCI_diff_norm',
       'precip_baseyr_Zaid', 'precip_targetyr_Zaid', 'precip_baseyr_Kharif',
       'precip_targetyr_Kharif', 'precip_baseyr_Rabi', 'precip_targetyr_Rabi'],
      dtype='object')


In [ ]:
df_main = df_main.drop(['Block', 'State', 'District'], axis=1)

In [ ]:
df_psm = df_main.copy()

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding
encoder_df = pd.DataFrame(encoder.fit_transform( df_main[['HSG']]).toarray())

#merge one-hot encoded columns back with original DataFrame
df_main =  df_main.join(encoder_df)

In [ ]:
df_main =  df_main.rename(columns={0:'HSG_3',1:'HSG_4', 2:'HSG_13', 3:'HSG_14'})
df_main.head()

,UID,lat,lon,Treatment,Asset ID,objectid,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,.geo,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm,ndmi_Zaid_base,ndmi_Zaid_base_norm,ndmi_Zaid_target,ndmi_Zaid_target_norm,Zaid_ndvi_diff,Zaid_ndvi_diff_norm,Zaid_ndmi_diff,Zaid_ndmi_diff_norm,Zaid_GCI_diff,Zaid_GCI_diff_norm,precip_baseyr_Zaid,precip_targetyr_Zaid,precip_baseyr_Kharif,precip_targetyr_Kharif,precip_baseyr_Rabi,precip_targetyr_Rabi,HSG_3,HSG_4,HSG_13,HSG_14,4
0,3734,24.391782,86.647062,1,34001162850,2005,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2018,0.109816,0.000000,0.113508,0.038989,0.342904,0.000000,0.376939,0.139014,-0.020645,0.452287,0.000055,0.679525,-0.003692,-0.038989,0.020700,0.227238,-0.034035,-0.139014,413.549218,373.426855,525.392053,385.226648,7.632749,102.324426,0.0,0.0,1.0,0.0,0.0
1,3734,24.391782,86.647062,1,34001162850,2005,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2019,0.184419,0.787747,0.113508,0.038989,0.511011,0.686629,0.376939,0.139014,-0.020645,0.452287,0.029239,0.999890,0.070911,0.748758,0.049884,0.547604,0.134072,0.547615,413.549218,398.951436,525.392053,734.113161,7.632749,87.216267,0.0,0.0,1.0,0.0,0.0
2,3734,24.391782,86.647062,1,34001162850,2005,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2020,0.140301,0.321897,0.113508,0.038989,0.454736,0.456775,0.376939,0.139014,-0.020645,0.452287,-0.038038,0.261357,0.026793,0.282908,-0.017393,-0.190930,0.077797,0.317761,413.549218,355.412280,525.392053,898.341936,7.632749,8.973775,0.0,0.0,1.0,0.0,0.0
3,3734,24.391782,86.647062,1,34001162850,2005,0.000000,2757.812358,2788.872635,39.261002,235.364174,66,2.398827,2.689599,13.0,6.2,0.58,15.0,"{""type"":""Point"",""coordinates"":[86.647062,24.39...",1.0,2016,2021,0.204510,0.999894,0.113508,0.038989,0.587723,0.999959,0.376939,0.139014,-0.020645,0.452287,-0.024894,0.405645,0.091002,0.960906,-0.004249,-0.046641,0.210784,0.860945,413.549218,202.874512,525.392053,624.465913,7.632749,112.464428,0.0,0.0,1.0,0.0,0.0
4,11503,22.683443,86.417664,1,34000979364,3171,103.653843,4283.513701,2289.282683,0.000000,127.948709,1,5.554252,2.807031,3.0,7.9,0.75,43.0,"{""type"":""Point"",""coordinates"":[86.417664,22.68...",1.0,2016,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476.438801,608.346255,644.703565,650.041761,4.589364,88.970089,1.0,0.0,0.0,0.0,0.0


In [ ]:
df_main.head()
df_main = df_main.drop(['built_in_2017', 'size', 'Total_Expenditure', 'pointInRegion', 'Asset Name', 'Block', 'District', 'Panchayat', 'block_coords', 'district_coords', 'state_coords', '.geo', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3'], axis=1, errors='ignore')
df_main.shape

(40000, 50)

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

df_clean = clean_dataset(df_main)

print(df_clean.shape, df_clean[df_clean['Treatment'] == 1.0].shape, df_clean[df_clean['Treatment'] == 0.0].shape)

(38816, 50) (9508, 50) (29308, 50)


In [ ]:
def normaliseFeatures(standard_scaler_vars, prec_vars, power_transform_vars):
  df_norm = df_psm.copy()
  scaler = preprocessing.StandardScaler()
  pt = preprocessing.PowerTransformer()
  for var in standard_scaler_vars:
    var_col = np.asarray(df_psm[var]).reshape(-1, 1)
    df_norm[var] = scaler.fit_transform(var_col)
  for var in power_transform_vars:
    var_col = np.asarray(df_psm[var]).reshape(-1, 1)
    df_norm[var] = pt.fit_transform(var_col)
  return df_norm

In [ ]:
standard_scaler_vars = ['elevation','slope']
prec_vars = [f'precip_baseyr_{season_name}'	,f'precip_targetyr_{season_name}']
soil_props = ['CEC', 'OC', 'pH']
drought_freq_vars = ['drought_freq_baseyr', 'drought_freq_targetyr']
# power_transform_vars = ['dist_closest_lin','dist_closest_river','dist_closest_road','dist_closest_upstream_forest','proximity_water', 'flow_accumulation'] + soil_props + prec_vars + drought_freq_vars
power_transform_vars = ['dist_closest_lin','dist_closest_river','dist_closest_upstream_forest','proximity_water', 'flow_accumulation'] + soil_props + prec_vars

In [ ]:
df_psm_norm = normaliseFeatures(standard_scaler_vars, prec_vars, power_transform_vars)

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column
encoder_df = pd.DataFrame(encoder.fit_transform( df_psm_norm[['HSG']]).toarray())

#merge one-hot encoded columns back with original DataFrame
df_psm_norm =  df_psm_norm.join(encoder_df)
df_psm_norm =  df_psm_norm.rename(columns={0:'HSG_3',1:'HSG_4', 2:'HSG_13', 3:'HSG_14'})
df_psm_norm.columns

Index([                         'UID',                          'lat',
                                'lon',                    'Treatment',
                           'Asset ID',                     'objectid',
                  'dist_closest_crop',             'dist_closest_lin',
                 'dist_closest_river', 'dist_closest_upstream_forest',
                          'elevation',            'flow_accumulation',
                    'proximity_water',                        'slope',
                                'HSG',                           'pH',
                                 'OC',                          'CEC',
                               '.geo',    'recent_cropping_intensity',
                          'base_year',                  'target_year',
                   'ndvi_Zaid_target',        'ndvi_Zaid_target_norm',
                     'ndvi_Zaid_base',          'ndvi_Zaid_base_norm',
                    'GCI_Zaid_target',         'GCI_Zaid_target_norm',
      

In [ ]:
drop_list = ['built_in_2017', 'size', 'water_visible', 'valid',	'.geo', 4]
df_psm_norm = df_psm_norm.drop(columns=drop_list, errors='ignore')
df_psm_norm.fillna(0, inplace=True)

In [ ]:
# add key to data
df_psm_norm['key'] = df_psm_norm['UID'].astype(int).astype(str) + '_' + df_psm_norm['base_year'].astype(int).astype(str) + '_' + df_psm_norm['target_year'].astype(int).astype(str)
print(df_psm_norm.shape)
# norm_global = df_psm_norm.copy()


(40000, 50)


In [ ]:
# metadata = ['Total_Expenditure', 'Asset ID', 'Asset Name', 'Block', 'District', 'Panchayat','UID','HSG','built_in_2017','size','water_visible','valid','recent_cropping_intensity','base_year','target_year', 'state_coords', 'district_coords', 'block_coords','.geo'] # depends on the data, remove everything except covariates
metadata = ['Total_Expenditure', 'Asset ID', 'Asset Name', 'Block', 'District', 'Panchayat','UID','HSG','recent_cropping_intensity','base_year','target_year', 'state_coords', 'district_coords', 'block_coords'] # depends on the data, remove everything except covariates
outcome_cols1 = [f'ndvi_{season_name}_target',	f'ndvi_{season_name}_target_norm',	f'ndvi_{season_name}_base',	f'ndvi_{season_name}_base_norm', f'{season_name}_ndvi_diff', f'{season_name}_ndvi_diff_norm']
outcome_cols2 = [f'ndmi_{season_name}_target',	f'ndmi_{season_name}_target_norm',	f'ndmi_{season_name}_base',	f'ndmi_{season_name}_base_norm', f'{season_name}_ndmi_diff', f'{season_name}_ndmi_diff_norm']
outcome_cols3 = [f'GCI_{season_name}_target',	f'GCI_{season_name}_target_norm',	f'GCI_{season_name}_base',	f'GCI_{season_name}_base_norm', f'{season_name}_GCI_diff', f'{season_name}_GCI_diff_norm']
# confounding_vars = ['dist_closest_lin', 'dist_closest_river', 'dist_closest_road', 'dist_closest_upstream_forest', 'elevation',
#  'slope', 'proximity_water','drainage_density','HSG_3','HSG_4','HSG_13','HSG_14','CEC','PH','OC', 'base_freq_baseyr', 'base_freq_targetyr', f'precip_baseyr_{season_name}', f'precip_targetyr_{season_name}']
confounding_vars = ['dist_closest_lin', 'dist_closest_river',  'dist_closest_upstream_forest', 'elevation', 'slope', 'proximity_water','HSG_3','HSG_4','HSG_13','HSG_14','CEC','PH','OC', 'drought_freq_baseyr', 'drought_freq_targetyr', 'flow_accumulation', f'precip_baseyr_{season_name}', f'precip_targetyr_{season_name}']

In [ ]:
df_psm_norm.columns

Index(['UID', 'lat', 'lon', 'Treatment', 'Asset ID', 'objectid',
       'dist_closest_crop', 'dist_closest_lin', 'dist_closest_river',
       'dist_closest_upstream_forest', 'elevation', 'flow_accumulation',
       'proximity_water', 'slope', 'HSG', 'pH', 'OC', 'CEC',
       'recent_cropping_intensity', 'base_year', 'target_year',
       'ndvi_Zaid_target', 'ndvi_Zaid_target_norm', 'ndvi_Zaid_base',
       'ndvi_Zaid_base_norm', 'GCI_Zaid_target', 'GCI_Zaid_target_norm',
       'GCI_Zaid_base', 'GCI_Zaid_base_norm', 'ndmi_Zaid_base',
       'ndmi_Zaid_base_norm', 'ndmi_Zaid_target', 'ndmi_Zaid_target_norm',
       'Zaid_ndvi_diff', 'Zaid_ndvi_diff_norm', 'Zaid_ndmi_diff',
       'Zaid_ndmi_diff_norm', 'Zaid_GCI_diff', 'Zaid_GCI_diff_norm',
       'precip_baseyr_Zaid', 'precip_targetyr_Zaid', 'precip_baseyr_Kharif',
       'precip_targetyr_Kharif', 'precip_baseyr_Rabi', 'precip_targetyr_Rabi',
       'HSG_3', 'HSG_4', 'HSG_13', 'HSG_14', 'key'],
      dtype='object')

In [ ]:
if psm_matched:
    dropcols = metadata + outcome_cols1 + outcome_cols2 + outcome_cols3
    psm = PsmPy(df_psm_norm, treatment='Treatment', indx='key', exclude = dropcols)
    psm.logistic_ps(balance = True)

In [ ]:
if psm_matched:
    p_scores = psm.predicted_data.copy()
    p_scores.set_index('key',inplace=True)
    p_scores.sort_index(inplace=True)
    p_scores = p_scores[['propensity_score','propensity_logit']]
    p_scores

In [ ]:
if psm_matched:
    df_psm_norm.set_index('key',inplace=True)
    dataset_psm = df_psm_norm.join(p_scores)
    dataset_psm

In [ ]:
def get_matching_pairs(treated_df, non_treated_df):
    treated_x = treated_df.values
    non_treated_x = non_treated_df.values
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='brute').fit(non_treated_x)
    distances, indices = nbrs.kneighbors(treated_x)
    indices = indices.reshape(indices.shape[0])
    matched = non_treated_df.iloc[indices]
    return matched

def get_base_target_year(key):
  parts = key.split('_')
  base_yr = parts[1]
  target_yr = parts[2]
  dict = {'base_yr': base_yr, 'target_yr':target_yr}
  return dict

Old code - below 2 cells not needed anymore

In [ ]:
df = dataset_psm.copy()
df.reset_index(inplace=True)
df

NameError: name 'dataset_psm' is not defined

In [ ]:
all_matchings = pd.DataFrame()
blocks_no_cp = set()
blocks_no_tp = set()
for block in df['block_coords'].unique():
    treated_df = df[(df['block_coords']==block) & (df['Treatment']==1)][['key','propensity_logit', 'base_year', 'target_year']]
    if len(treated_df)>0:
        for index, row in treated_df.iterrows():
            non_treated_df = df[(df['block_coords']==block) & (df['Treatment']==0) & (df['base_year']== row['base_year']) & (df['target_year']== row['target_year'])][['key','propensity_logit', 'base_year', 'target_year']]
            non_treated_df.replace([np.inf, -np.inf], np.nan, inplace=True)
            non_treated_df.fillna(0, inplace=True)
            if len(non_treated_df)>0:
                dict = {'propensity_logit': row['propensity_logit']}
                row_df = pd.DataFrame(dict, index = [row['key']])
                row_df.replace([np.inf, -np.inf], np.nan, inplace=True)
                row_df.fillna(0, inplace=True)
                non_treated_df = non_treated_df[['key','propensity_logit']].set_index('key')
                matched_df = get_matching_pairs(row_df, non_treated_df)
                matching = pd.DataFrame({'key': row_df.index, 'matched_key': matched_df.index})
                all_matchings = pd.concat([all_matchings, matching], axis=0)
            else:
                blocks_no_cp.add(block)
                # print('block: ',block)
                # print('len(treated_df): ',len(treated_df))
                # print('len(non_treated_df): ',len(non_treated_df))
    else:
        blocks_no_tp.add(block)
        # print('block: ',block)
        # print('len(treated_df): ',len(treated_df))
        # print('len(non_treated_df): ',len(non_treated_df))

KeyError: 'block_coords'

In [ ]:
print(len(blocks_no_cp))
print(len(blocks_no_tp))

0
0


In [ ]:
# export matchings if needed
# all_matchings.reset_index(inplace=True)
all_matchings

,key,matched_key
0,3734_2016_2018,78326_2016_2018
0,3734_2016_2019,90474_2016_2019
0,3734_2016_2020,47897_2016_2020
0,3734_2016_2021,86690_2016_2021
0,6424_2016_2018,36598_2016_2018
...,...,...
0,19839_2016_2021,91159_2016_2021
0,22720_2016_2018,55000_2016_2018
0,22720_2016_2019,67679_2016_2019
0,22720_2016_2020,95591_2016_2020


In [ ]:
print(len(all_matchings['key'].unique()))
print(len(all_matchings['matched_key'].unique()))

9508
7387


In [ ]:
def get_UID(s):
    return int(s.split('_')[0])

In [ ]:
if psm_matched:
    matchings = all_matchings.copy()
    treated_uids = set()
    control_uids = set()
    for index, row in matchings.iterrows():
        treated_uid = get_UID(row['key'])
        control_uid = get_UID(row['matched_key'])
        # print(treated_uid, control_uid)
        treated_uids.add(treated_uid)
        control_uids.add(control_uid)
    treated_uids = list(treated_uids)
    control_uids = list(control_uids)

    print(len(treated_uids))
    print(len(control_uids))

    # Resample data points
    df_sample_treated = df_clean[df_clean['Treatment'] == 1]
    df_sample_control = df_clean[df_clean['Treatment'] == 0]
    df_sample_treated = df_sample_treated.sort_values('UID')
    df_sample_control = df_sample_control.sort_values('UID')

    print(df_sample_treated.shape)
    print(df_sample_control.shape)

    df_sampled_treated = df_sample_treated[df_sample_treated['UID'].isin(treated_uids)]
    df_sampled_control = df_sample_control[df_sample_control['UID'].isin(control_uids)]


    print(df_sampled_treated.shape)
    print(df_sampled_control.shape)

    df_sampled = pd.concat([df_sampled_treated, df_sampled_control], axis=0, ignore_index=True)
    df_sampled = df_sampled.sample(frac=1, random_state=7359) # to shuffle the rows
    df_sampled.shape

In [ ]:
if not psm_matched:
    df_clean = df_clean.drop(4, axis = 1, errors='ignore')
    print(df_clean.shape, df_clean[df_clean['Treatment'] == 1.0].shape, df_clean[df_clean['Treatment'] == 0.0].shape)
    treated_count = df_clean[df_clean['Treatment'] == 1.0].shape[0]
    control_count = df_clean[df_clean['Treatment'] == 0.0].shape[0]
    req_fr = (3.0 * treated_count / (1 + control_count))
    print(req_fr)

(38816, 49) (9508, 49) (29308, 49)
0.9732164181650688


In [ ]:
if not psm_matched:
    # Resample data points. Number of control points = 3*number of treated points
    df_sample_treated = df_clean[df_clean['Treatment'] == 1]
    df_sample_control = df_clean[df_clean['Treatment'] == 0]
    df_sample_treated = df_sample_treated.sort_values('UID')
    df_sample_control = df_sample_control.sort_values('UID')

    df_sample_control = df_sample_control.sample(frac=req_fr, random_state=153) # num of control = 3*number of treated points
    df_sampled = pd.concat([df_sample_treated, df_sample_control], axis=0, ignore_index=True)
    df_sampled = df_sampled.sample(frac=1, random_state=7359) # to shuffle the rows
    df_sampled.shape

    print(df_sampled.shape, df_sampled[df_sampled['Treatment'] == 1.0].shape, df_sampled[df_sampled['Treatment'] == 0.0].shape)

(38031, 49) (9508, 49) (28523, 49)


In [ ]:
if psm_matched:
    df_sampled.to_csv(f"Allyears_NDVI_df_{season_name}_{str_date}_psm_matched.csv", index=False)
else:
    df_sampled.to_csv(f"Allyears_NDVI_df_{season_name}_{str_date}_random.csv", index=False)

#Step 7 - DiD - RQ-1

In [ ]:
# str_date = "Apr-15-2024" # if want to use old dumped df
today = date.today()
str_date = today.strftime("%b-%d-%Y")

season = 2 # 1 (Kharif), 2 (Rabi), 3 (Zaid)
# dataset = 1 # 1 (built in 2017), 2 (valid farm ponds)
# psm_matched = 1 # 0/1 if you want psm matched dataset. If 0, then #cp = 3*#tp

In [ ]:
if season == 1:
    season_name = "Kharif"
elif season == 2:
    season_name = "Rabi"
elif season == 3:
    season_name = "Zaid"
else:
    raise Exception("Please set season value")
season_name

'Zaid'

In [ ]:

df_main = pd.read_csv(f"Allyears_NDVI_df_{season_name}_{str_date}_random.csv")


In [ ]:
def normaliseFeatures(standard_scaler_vars, prec_vars, power_transform_vars):
  df_norm = df_main.copy()
  scaler = preprocessing.StandardScaler()
  pt = preprocessing.PowerTransformer()
  for var in standard_scaler_vars:
    var_col = np.asarray(df_main[var]).reshape(-1, 1)
    df_norm[var] = scaler.fit_transform(var_col)
  for var in power_transform_vars:
    var_col = np.asarray(df_main[var]).reshape(-1, 1)
    df_norm[var] = pt.fit_transform(var_col)
  return df_norm

In [ ]:
standard_scaler_vars = ['elevation','slope']
prec_vars = [f'precip_baseyr_{season_name}'	,f'precip_targetyr_{season_name}']
if season_name == "Zaid" or season_name == "Rabi":
    prec_vars.append('precip_baseyr_Rabi')
    prec_vars.append('precip_targetyr_Rabi')
if season_name == "Zaid":
    prec_vars.append('precip_baseyr_Zaid')
    prec_vars.append('precip_targetyr_Zaid')

soil_props = ['CEC', 'OC', 'pH']
drought_freq_vars = ['drought_freq_baseyr', 'drought_freq_targetyr']
# power_transform_vars = ['dist_closest_lin','dist_closest_river','dist_closest_road','dist_closest_upstream_forest','proximity_water', 'flow_accumulation'] + soil_props + prec_vars + drought_freq_vars
power_transform_vars = ['dist_closest_lin','dist_closest_river','dist_closest_upstream_forest','proximity_water', 'flow_accumulation'] + soil_props + prec_vars

In [ ]:
df = normaliseFeatures(standard_scaler_vars, prec_vars, power_transform_vars)

In [ ]:
df.columns

Index(['UID', 'lat', 'lon', 'Treatment', 'Asset ID', 'objectid',
       'dist_closest_crop', 'dist_closest_lin', 'dist_closest_river',
       'dist_closest_upstream_forest', 'elevation', 'flow_accumulation',
       'proximity_water', 'slope', 'HSG', 'pH', 'OC', 'CEC',
       'recent_cropping_intensity', 'base_year', 'target_year',
       'ndvi_Zaid_target', 'ndvi_Zaid_target_norm', 'ndvi_Zaid_base',
       'ndvi_Zaid_base_norm', 'GCI_Zaid_target', 'GCI_Zaid_target_norm',
       'GCI_Zaid_base', 'GCI_Zaid_base_norm', 'ndmi_Zaid_base',
       'ndmi_Zaid_base_norm', 'ndmi_Zaid_target', 'ndmi_Zaid_target_norm',
       'Zaid_ndvi_diff', 'Zaid_ndvi_diff_norm', 'Zaid_ndmi_diff',
       'Zaid_ndmi_diff_norm', 'Zaid_GCI_diff', 'Zaid_GCI_diff_norm',
       'precip_baseyr_Zaid', 'precip_targetyr_Zaid', 'precip_baseyr_Kharif',
       'precip_targetyr_Kharif', 'precip_baseyr_Rabi', 'precip_targetyr_Rabi',
       'HSG_3', 'HSG_4', 'HSG_13', 'HSG_14'],
      dtype='object')

In [ ]:
drop_list = ['built_in_2017', 'size', 'water_visible', 'valid',	'.geo', 4, ]
df = df.drop(columns=drop_list, errors='ignore')
df.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

df = clean_dataset(df)

In [ ]:

df.to_csv(f"Allyears_NDVI_df_{season_name}_normalised_{str_date}_random.csv", index=False)
# df.to_csv(f"Allyears_NDVI_df_{season_name}_normalised_{str_date}.csv", index=False)

In [ ]:

df = pd.read_csv(f"Allyears_NDVI_df_{season_name}_normalised_{str_date}_random.csv")
# df = pd.read_csv(f"Allyears_NDVI_df_{season_name}_normalised_{str_date}.csv")

In [ ]:
df.shape
df[df["Treatment"] == 0]


,UID,lat,lon,Treatment,Asset ID,objectid,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm,ndmi_Zaid_base,ndmi_Zaid_base_norm,ndmi_Zaid_target,ndmi_Zaid_target_norm,Zaid_ndvi_diff,Zaid_ndvi_diff_norm,Zaid_ndmi_diff,Zaid_ndmi_diff_norm,Zaid_GCI_diff,Zaid_GCI_diff_norm,precip_baseyr_Zaid,precip_targetyr_Zaid,precip_baseyr_Kharif,precip_targetyr_Kharif,precip_baseyr_Rabi,precip_targetyr_Rabi,HSG_3,HSG_4,HSG_13,HSG_14
0,68372.0,24.047945,87.418788,0.0,0.0,2484.0,0.000000,0.340833,-0.208665,1.578315,-0.853890,0.069238,0.533669,0.497028,3.0,-0.181999,-0.805504,-0.806046,1.0,2016.0,2018.0,0.162812,0.618860,0.113904,0.000000,0.467907,0.624161,0.352361,0.000000,-0.028684,0.000000,-0.020370,0.071376,0.048908,0.618860,0.008314,0.071376,0.115545,0.624161,0.495711,0.309156,528.539276,403.687234,-0.217158,0.138852,1.0,0.0,0.0,0.0
2,56781.0,24.358133,86.628900,0.0,0.0,2029.0,0.000000,1.220890,0.016867,0.810266,0.192344,1.707868,0.129943,-0.238765,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2019.0,0.124008,0.165530,0.119885,0.096156,0.316620,0.000000,0.375374,0.330723,-0.006127,0.248238,0.080761,0.999913,0.004123,0.069374,0.086888,0.751675,-0.058755,-0.330723,-0.477342,-0.093826,559.390961,728.953635,0.497301,0.288390,1.0,0.0,0.0,0.0
5,41054.0,23.894603,87.023709,0.0,0.0,2090.0,0.000000,-2.268891,0.802688,0.634099,-0.444898,-0.974124,0.940516,0.327600,3.0,-0.042371,-1.164288,-0.630983,1.0,2016.0,2019.0,0.119960,0.032466,0.146888,0.431123,0.342426,0.000000,0.438138,0.497739,-0.039786,0.000000,0.043181,0.999879,-0.026928,-0.398657,0.082967,0.999879,-0.095712,-0.497739,0.533002,1.306732,640.115773,634.424335,-0.058130,-0.230420,1.0,0.0,0.0,0.0
7,70558.0,23.154571,85.814757,0.0,0.0,3163.0,0.000000,0.422337,-0.440427,1.609979,-0.031391,0.069238,1.038966,-0.021722,3.0,-2.038057,1.110082,-0.317867,1.0,2016.0,2021.0,0.131575,0.634586,0.117177,0.244546,0.403146,0.501262,0.412044,0.569442,-0.052605,0.127729,0.003147,0.922855,0.014397,0.390039,0.055753,0.795126,-0.008898,-0.068180,-0.628829,-1.259315,650.082392,744.566938,0.036722,0.903687,1.0,0.0,0.0,0.0
8,35800.0,24.451917,87.015085,0.0,0.0,2480.0,0.000000,-1.261386,1.175253,0.867219,-0.126318,-0.309931,-1.267857,0.101756,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2018.0,0.144622,0.320583,0.119016,0.000000,0.462844,0.425326,0.381789,0.000000,0.007315,0.653543,0.019448,0.828687,0.025606,0.320583,0.012133,0.175144,0.081056,0.425326,0.860314,-0.391449,572.521621,376.784501,0.525976,0.353872,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38026,92170.0,24.219074,86.971966,0.0,0.0,2033.0,0.000000,0.258556,-1.527394,-1.742896,0.228772,-0.974124,-0.287323,0.230431,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2021.0,0.155454,0.425693,0.151648,0.319053,0.420705,0.000000,0.481344,0.795946,-0.039083,0.086459,0.018367,0.753959,0.003806,0.106640,0.057451,0.667500,-0.060639,-0.795946,0.644579,-1.238261,591.230568,676.289204,0.520610,0.484840,1.0,0.0,0.0,0.0
38027,44148.0,24.419039,87.761855,0.0,0.0,2526.0,0.000000,-0.215378,-1.897305,0.300256,-1.259543,1.026313,1.280876,-0.181622,14.0,-0.042371,0.275036,-0.317867,1.0,2016.0,2019.0,0.136766,0.617641,0.113471,0.280349,0.328883,0.581467,0.311569,0.477168,0.043553,0.650996,0.053111,0.758285,0.023295,0.337292,0.009558,0.107290,0.017314,0.104299,0.493083,0.837685,634.060997,667.832966,0.875087,0.188334,0.0,0.0,0.0,1.0
38028,31652.0,24.146041,86.248912,0.0,0.0,2054.0,0.000000,0.300507,0.305377,1.022950,0.228507,1.566448,0.475858,-0.696016,4.0,0.374073,1.086967,0.440463,1.0,2016.0,2021.0,0.133347,0.238248,0.122289,0.062866,0

In [ ]:
norm = df.copy()
norm['key'] = norm['UID'].astype(int).astype(str) + '_' + norm['base_year'].astype(int).astype(str) + '_' + norm['target_year'].astype(int).astype(str)
print(norm.shape)
norm_global = norm.copy()


(38031, 50)


In [ ]:
# metadata = ['Total_Expenditure', 'Asset ID', 'Asset Name', 'Block', 'District', 'Panchayat','UID','HSG','built_in_2017','size','water_visible','valid','recent_cropping_intensity','base_year','target_year', 'state_coords', 'district_coords', 'block_coords','.geo'] # depends on the data, remove everything except covariates
# metadata = ['Asset ID', 'Asset Name', 'Block', 'District', 'Panchayat','UID','HSG','recent_cropping_intensity','base_year','target_year', 'state_coords', 'district_coords', 'block_coords'] # depends on the data, remove everything except covariates
metadata2 = ['UID','objectid','recent_cropping_intensity','base_year','target_year','Asset ID', 'HSG'] # depends on the data, remove everything except covariates
outcome_cols1 = [f'ndvi_{season_name}_target',	f'ndvi_{season_name}_target_norm',	f'ndvi_{season_name}_base',	f'ndvi_{season_name}_base_norm', f'{season_name}_ndvi_diff', f'{season_name}_ndvi_diff_norm']
outcome_cols2 = [f'ndmi_{season_name}_target',	f'ndmi_{season_name}_target_norm',	f'ndmi_{season_name}_base',	f'ndmi_{season_name}_base_norm', f'{season_name}_ndmi_diff', f'{season_name}_ndmi_diff_norm']
outcome_cols3 = [f'GCI_{season_name}_target',	f'GCI_{season_name}_target_norm',	f'GCI_{season_name}_base',	f'GCI_{season_name}_base_norm', f'{season_name}_GCI_diff', f'{season_name}_GCI_diff_norm']
# confounding_vars = ['dist_closest_lin', 'dist_closest_river', 'dist_closest_road', 'dist_closest_upstream_forest', 'elevation',
#  'slope', 'proximity_water','drainage_density','HSG_3','HSG_4','HSG_13','HSG_14','CEC','PH','OC', 'base_freq_baseyr', 'base_freq_targetyr', f'precip_baseyr_{season_name}', f'precip_targetyr_{season_name}']
confounding_vars = ['dist_closest_lin', 'dist_closest_river', 'dist_closest_road', 'dist_closest_upstream_forest', 'elevation', 'slope', 'proximity_water','drainage_density','HSG_3','HSG_4','HSG_13','HSG_14','CEC','PH','OC', 'flow_accumulation', f'precip_baseyr_{season_name}', f'precip_targetyr_{season_name}']

In [ ]:
# dropcols = metadata + outcome_cols1 + outcome_cols2 + outcome_cols3
dropcols = metadata2 + outcome_cols1 + outcome_cols2 + outcome_cols3
print(dropcols)
# dropcols = metadata2 + outcome_cols1 + outcome_cols3
psm = PsmPy(norm, treatment='Treatment', indx='key', exclude = dropcols)
psm.logistic_ps(balance = True)

['UID', 'objectid', 'recent_cropping_intensity', 'base_year', 'target_year', 'Asset ID', 'HSG', 'ndvi_Zaid_target', 'ndvi_Zaid_target_norm', 'ndvi_Zaid_base', 'ndvi_Zaid_base_norm', 'Zaid_ndvi_diff', 'Zaid_ndvi_diff_norm', 'ndmi_Zaid_target', 'ndmi_Zaid_target_norm', 'ndmi_Zaid_base', 'ndmi_Zaid_base_norm', 'Zaid_ndmi_diff', 'Zaid_ndmi_diff_norm', 'GCI_Zaid_target', 'GCI_Zaid_target_norm', 'GCI_Zaid_base', 'GCI_Zaid_base_norm', 'Zaid_GCI_diff', 'Zaid_GCI_diff_norm']


In [ ]:
p_scores = psm.predicted_data.copy()
# p_scores.set_index('key',inplace=True)
# p_scores.sort_index(inplace=True)
p_scores = p_scores[['propensity_score','propensity_logit', 'key']]
p_scores

,propensity_score,propensity_logit,key
0,0.567868,0.273157,12847_2016_2020
1,0.399919,-0.405802,13666_2016_2020
2,0.553951,0.216645,6536_2016_2021
3,0.608376,0.440489,14545_2016_2019
4,0.593060,0.376629,18663_2016_2018
...,...,...,...
38026,0.671552,0.715212,92170_2016_2021
38027,0.269871,-0.995275,44148_2016_2019
38028,0.371871,-0.524198,31652_2016_2021
38029,0.434182,-0.264808,72333_2016_2021


In [ ]:
# norm.set_index('key',inplace=True)
# norm.sort_index(inplace=True)
norm = norm.drop_duplicates(subset='key')
dataset_psm = norm.merge(p_scores, on='key')
dataset_psm
# check propensity_score describe
# propensity_logit
# data getting duplicated somewhere
#  HSG needs to be dropped

,UID,lat,lon,Treatment,Asset ID,objectid,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm,ndmi_Zaid_base,ndmi_Zaid_base_norm,ndmi_Zaid_target,ndmi_Zaid_target_norm,Zaid_ndvi_diff,Zaid_ndvi_diff_norm,Zaid_ndmi_diff,Zaid_ndmi_diff_norm,Zaid_GCI_diff,Zaid_GCI_diff_norm,precip_baseyr_Zaid,precip_targetyr_Zaid,precip_baseyr_Kharif,precip_targetyr_Kharif,precip_baseyr_Rabi,precip_targetyr_Rabi,HSG_3,HSG_4,HSG_13,HSG_14,key,propensity_score,propensity_logit
0,68372.0,24.047945,87.418788,0.0,0.000000e+00,2484.0,0.000000,0.340833,-0.208665,1.578315,-0.853890,0.069238,0.533669,0.497028,3.0,-0.181999,-0.805504,-0.806046,1.0,2016.0,2018.0,0.162812,0.618860,0.113904,0.000000,0.467907,0.624161,0.352361,0.000000,-0.028684,0.000000,-0.020370,0.071376,0.048908,0.618860,0.008314,0.071376,0.115545,0.624161,0.495711,0.309156,528.539276,403.687234,-0.217158,0.138852,1.0,0.0,0.0,0.0,68372_2016_2018,0.398460,-0.411887
1,12847.0,24.352310,85.982447,1.0,3.400089e+10,2028.0,0.000000,0.304837,-0.324090,-0.357287,0.768561,-0.974124,0.505901,-0.527148,4.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.116945,0.562381,0.109419,0.413036,0.408873,0.340276,0.411786,0.358533,-0.031376,0.469570,0.025851,0.999907,0.007527,0.149345,0.057228,0.530337,-0.002913,-0.018256,-2.476260,0.137784,601.958832,877.048239,0.163029,-1.464819,0.0,1.0,0.0,0.0,12847_2016_2020,0.567868,0.273157
2,56781.0,24.358133,86.628900,0.0,0.000000e+00,2029.0,0.000000,1.220890,0.016867,0.810266,0.192344,1.707868,0.129943,-0.238765,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2019.0,0.124008,0.165530,0.119885,0.096156,0.316620,0.000000,0.375374,0.330723,-0.006127,0.248238,0.080761,0.999913,0.004123,0.069374,0.086888,0.751675,-0.058755,-0.330723,-0.477342,-0.093826,559.390961,728.953635,0.497301,0.288390,1.0,0.0,0.0,0.0,56781_2016_2019,0.400063,-0.405201
3,13666.0,24.079788,86.292278,1.0,3.400097e+10,2054.0,60.295648,-1.444907,-0.533738,-0.935615,-0.003682,0.069238,1.159696,-0.250703,14.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.164903,0.674914,0.133089,0.000000,0.493146,0.639235,0.404250,0.000000,-0.009286,0.286936,-0.014350,0.232087,0.031814,0.674914,-0.005064,-0.054849,0.088896,0.639235,-0.976481,0.318754,577.221264,865.177002,-0.146142,-1.541058,0.0,0.0,0.0,1.0,13666_2016_2020,0.399919,-0.405802
4,6536.0,23.981077,86.476573,1.0,3.400120e+10,2064.0,0.000000,-1.027938,0.872949,-0.275503,-0.047996,-0.309931,0.001460,-0.207741,3.0,1.737654,1.685346,-2.148870,2.0,2016.0,2021.0,0.199449,0.999906,0.148504,0.521180,0.542436,0.999964,0.417131,0.544369,0.017200,0.485149,0.034164,0.671976,0.050946,0.478726,0.016964,0.186827,0.125306,0.455595,-0.336289,-1.296969,621.096923,674.731772,0.501513,0.349991,1.0,0.0,0.0,0.0,6536_2016_2021,0.553951,0.216645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38026,92170.0,24.219074,86.971966,0.0,0.000000e+00,2033.0,0.000000,0.258556,-1.527394,-1.742896,0.228772,-0.974124,-0.287323,0.230431,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2021.0,0.155454,0.425693,0.151648,0.319053,0.420705,0.000000,0.481344,0.795946,-0.039083,0.086459,0.018367,0.753959,0.003806,0.106640,0.057451,0.667500,-0.060639,-0.795946,0.644579,-1.238261,591.230568,676.289204,0.520610,0.484840,1.0,0.0,0.0,0.0,92170_2016_2021,0.671552,0.715212
38027,44148.0,24.419039,87.761855,0.0,0.000000e+00,2526.0,0.000000,-0.215378,-1.897305,0.300256,-1.259543,1.026313,1.280876,-0.181622,14.0,-0.042371,0.275036,-0.317867,1.0,2016.0,2019.0,0.136766,0.617641,0.113471,0.280349,0.328883,0.581467,0.311569,0.477168,0.043553,0.650996,0.053111,0.758285,0.023295,0.33

In [ ]:
dataset_psm = dataset_psm.drop_duplicates(subset='key')
dataset_psm

,UID,lat,lon,Treatment,Asset ID,objectid,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm,ndmi_Zaid_base,ndmi_Zaid_base_norm,ndmi_Zaid_target,ndmi_Zaid_target_norm,Zaid_ndvi_diff,Zaid_ndvi_diff_norm,Zaid_ndmi_diff,Zaid_ndmi_diff_norm,Zaid_GCI_diff,Zaid_GCI_diff_norm,precip_baseyr_Zaid,precip_targetyr_Zaid,precip_baseyr_Kharif,precip_targetyr_Kharif,precip_baseyr_Rabi,precip_targetyr_Rabi,HSG_3,HSG_4,HSG_13,HSG_14,key,propensity_score,propensity_logit
0,68372.0,24.047945,87.418788,0.0,0.000000e+00,2484.0,0.000000,0.340833,-0.208665,1.578315,-0.853890,0.069238,0.533669,0.497028,3.0,-0.181999,-0.805504,-0.806046,1.0,2016.0,2018.0,0.162812,0.618860,0.113904,0.000000,0.467907,0.624161,0.352361,0.000000,-0.028684,0.000000,-0.020370,0.071376,0.048908,0.618860,0.008314,0.071376,0.115545,0.624161,0.495711,0.309156,528.539276,403.687234,-0.217158,0.138852,1.0,0.0,0.0,0.0,68372_2016_2018,0.398460,-0.411887
1,12847.0,24.352310,85.982447,1.0,3.400089e+10,2028.0,0.000000,0.304837,-0.324090,-0.357287,0.768561,-0.974124,0.505901,-0.527148,4.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.116945,0.562381,0.109419,0.413036,0.408873,0.340276,0.411786,0.358533,-0.031376,0.469570,0.025851,0.999907,0.007527,0.149345,0.057228,0.530337,-0.002913,-0.018256,-2.476260,0.137784,601.958832,877.048239,0.163029,-1.464819,0.0,1.0,0.0,0.0,12847_2016_2020,0.567868,0.273157
2,56781.0,24.358133,86.628900,0.0,0.000000e+00,2029.0,0.000000,1.220890,0.016867,0.810266,0.192344,1.707868,0.129943,-0.238765,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2019.0,0.124008,0.165530,0.119885,0.096156,0.316620,0.000000,0.375374,0.330723,-0.006127,0.248238,0.080761,0.999913,0.004123,0.069374,0.086888,0.751675,-0.058755,-0.330723,-0.477342,-0.093826,559.390961,728.953635,0.497301,0.288390,1.0,0.0,0.0,0.0,56781_2016_2019,0.400063,-0.405201
3,13666.0,24.079788,86.292278,1.0,3.400097e+10,2054.0,60.295648,-1.444907,-0.533738,-0.935615,-0.003682,0.069238,1.159696,-0.250703,14.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.164903,0.674914,0.133089,0.000000,0.493146,0.639235,0.404250,0.000000,-0.009286,0.286936,-0.014350,0.232087,0.031814,0.674914,-0.005064,-0.054849,0.088896,0.639235,-0.976481,0.318754,577.221264,865.177002,-0.146142,-1.541058,0.0,0.0,0.0,1.0,13666_2016_2020,0.399919,-0.405802
4,6536.0,23.981077,86.476573,1.0,3.400120e+10,2064.0,0.000000,-1.027938,0.872949,-0.275503,-0.047996,-0.309931,0.001460,-0.207741,3.0,1.737654,1.685346,-2.148870,2.0,2016.0,2021.0,0.199449,0.999906,0.148504,0.521180,0.542436,0.999964,0.417131,0.544369,0.017200,0.485149,0.034164,0.671976,0.050946,0.478726,0.016964,0.186827,0.125306,0.455595,-0.336289,-1.296969,621.096923,674.731772,0.501513,0.349991,1.0,0.0,0.0,0.0,6536_2016_2021,0.553951,0.216645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38026,92170.0,24.219074,86.971966,0.0,0.000000e+00,2033.0,0.000000,0.258556,-1.527394,-1.742896,0.228772,-0.974124,-0.287323,0.230431,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2021.0,0.155454,0.425693,0.151648,0.319053,0.420705,0.000000,0.481344,0.795946,-0.039083,0.086459,0.018367,0.753959,0.003806,0.106640,0.057451,0.667500,-0.060639,-0.795946,0.644579,-1.238261,591.230568,676.289204,0.520610,0.484840,1.0,0.0,0.0,0.0,92170_2016_2021,0.671552,0.715212
38027,44148.0,24.419039,87.761855,0.0,0.000000e+00,2526.0,0.000000,-0.215378,-1.897305,0.300256,-1.259543,1.026313,1.280876,-0.181622,14.0,-0.042371,0.275036,-0.317867,1.0,2016.0,2019.0,0.136766,0.617641,0.113471,0.280349,0.328883,0.581467,0.311569,0.477168,0.043553,0.650996,0.053111,0.758285,0.023295,0.33

In [ ]:
def get_matching_pairs(treated_df, non_treated_df):
    treated_x = treated_df.values
    non_treated_x = non_treated_df.values
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='brute').fit(non_treated_x)
    distances, indices = nbrs.kneighbors(treated_x)
    indices = indices.reshape(indices.shape[0])
    matched = non_treated_df.iloc[indices]
    return matched

In [ ]:
def plot_hist (dist, label):
    mean = dist.mean()
    plt.hist(dist, bins=100, label=label)
    plt.axvline(mean, color='k', linestyle='dashed', label='Mean = '+str(mean))
    plt.legend()
    plt.show()

In [ ]:
df = dataset_psm.copy()
# drop rows with inf
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
# df.reset_index(inplace=True)
df
# print(df[(df['Treatment']==1)])

,UID,lat,lon,Treatment,Asset ID,objectid,dist_closest_crop,dist_closest_lin,dist_closest_river,dist_closest_upstream_forest,elevation,flow_accumulation,proximity_water,slope,HSG,pH,OC,CEC,recent_cropping_intensity,base_year,target_year,ndvi_Zaid_target,ndvi_Zaid_target_norm,ndvi_Zaid_base,ndvi_Zaid_base_norm,GCI_Zaid_target,GCI_Zaid_target_norm,GCI_Zaid_base,GCI_Zaid_base_norm,ndmi_Zaid_base,ndmi_Zaid_base_norm,ndmi_Zaid_target,ndmi_Zaid_target_norm,Zaid_ndvi_diff,Zaid_ndvi_diff_norm,Zaid_ndmi_diff,Zaid_ndmi_diff_norm,Zaid_GCI_diff,Zaid_GCI_diff_norm,precip_baseyr_Zaid,precip_targetyr_Zaid,precip_baseyr_Kharif,precip_targetyr_Kharif,precip_baseyr_Rabi,precip_targetyr_Rabi,HSG_3,HSG_4,HSG_13,HSG_14,key,propensity_score,propensity_logit
0,68372.0,24.047945,87.418788,0.0,0.000000e+00,2484.0,0.000000,0.340833,-0.208665,1.578315,-0.853890,0.069238,0.533669,0.497028,3.0,-0.181999,-0.805504,-0.806046,1.0,2016.0,2018.0,0.162812,0.618860,0.113904,0.000000,0.467907,0.624161,0.352361,0.000000,-0.028684,0.000000,-0.020370,0.071376,0.048908,0.618860,0.008314,0.071376,0.115545,0.624161,0.495711,0.309156,528.539276,403.687234,-0.217158,0.138852,1.0,0.0,0.0,0.0,68372_2016_2018,0.398460,-0.411887
1,12847.0,24.352310,85.982447,1.0,3.400089e+10,2028.0,0.000000,0.304837,-0.324090,-0.357287,0.768561,-0.974124,0.505901,-0.527148,4.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.116945,0.562381,0.109419,0.413036,0.408873,0.340276,0.411786,0.358533,-0.031376,0.469570,0.025851,0.999907,0.007527,0.149345,0.057228,0.530337,-0.002913,-0.018256,-2.476260,0.137784,601.958832,877.048239,0.163029,-1.464819,0.0,1.0,0.0,0.0,12847_2016_2020,0.567868,0.273157
2,56781.0,24.358133,86.628900,0.0,0.000000e+00,2029.0,0.000000,1.220890,0.016867,0.810266,0.192344,1.707868,0.129943,-0.238765,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2019.0,0.124008,0.165530,0.119885,0.096156,0.316620,0.000000,0.375374,0.330723,-0.006127,0.248238,0.080761,0.999913,0.004123,0.069374,0.086888,0.751675,-0.058755,-0.330723,-0.477342,-0.093826,559.390961,728.953635,0.497301,0.288390,1.0,0.0,0.0,0.0,56781_2016_2019,0.400063,-0.405201
3,13666.0,24.079788,86.292278,1.0,3.400097e+10,2054.0,60.295648,-1.444907,-0.533738,-0.935615,-0.003682,0.069238,1.159696,-0.250703,14.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.164903,0.674914,0.133089,0.000000,0.493146,0.639235,0.404250,0.000000,-0.009286,0.286936,-0.014350,0.232087,0.031814,0.674914,-0.005064,-0.054849,0.088896,0.639235,-0.976481,0.318754,577.221264,865.177002,-0.146142,-1.541058,0.0,0.0,0.0,1.0,13666_2016_2020,0.399919,-0.405802
4,6536.0,23.981077,86.476573,1.0,3.400120e+10,2064.0,0.000000,-1.027938,0.872949,-0.275503,-0.047996,-0.309931,0.001460,-0.207741,3.0,1.737654,1.685346,-2.148870,2.0,2016.0,2021.0,0.199449,0.999906,0.148504,0.521180,0.542436,0.999964,0.417131,0.544369,0.017200,0.485149,0.034164,0.671976,0.050946,0.478726,0.016964,0.186827,0.125306,0.455595,-0.336289,-1.296969,621.096923,674.731772,0.501513,0.349991,1.0,0.0,0.0,0.0,6536_2016_2021,0.553951,0.216645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38026,92170.0,24.219074,86.971966,0.0,0.000000e+00,2033.0,0.000000,0.258556,-1.527394,-1.742896,0.228772,-0.974124,-0.287323,0.230431,3.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2021.0,0.155454,0.425693,0.151648,0.319053,0.420705,0.000000,0.481344,0.795946,-0.039083,0.086459,0.018367,0.753959,0.003806,0.106640,0.057451,0.667500,-0.060639,-0.795946,0.644579,-1.238261,591.230568,676.289204,0.520610,0.484840,1.0,0.0,0.0,0.0,92170_2016_2021,0.671552,0.715212
38027,44148.0,24.419039,87.761855,0.0,0.000000e+00,2526.0,0.000000,-0.215378,-1.897305,0.300256,-1.259543,1.026313,1.280876,-0.181622,14.0,-0.042371,0.275036,-0.317867,1.0,2016.0,2019.0,0.136766,0.617641,0.113471,0.280349,0.328883,0.581467,0.311569,0.477168,0.043553,0.650996,0.053111,0.758285,0.023295,0.33

In [ ]:
def get_base_target_year(key):
  parts = key.split('_')
  base_yr = parts[1]
  target_yr = parts[2]
  dict = {'base_yr': base_yr, 'target_yr':target_yr}
  return dict

In [ ]:
get_base_target_year('13123_2018_2022')

{'base_yr': '2018', 'target_yr': '2022'}

In [ ]:
all_matchings = pd.DataFrame()
blocks_no_cp = set()
blocks_no_tp = set()

for baseyear in df['base_year'].unique():
  treated_df = df[(df['base_year'] == baseyear) & (df['Treatment']==1)][['key','propensity_logit', 'base_year', 'target_year']]
#   treated_df = treated_df.reset_index()
  if len(treated_df)>0:
    for index, row in treated_df.iterrows():
      non_treated_df = df[(df['base_year'] == baseyear) & (df['Treatment']==0) & (df['target_year']== row['target_year'])][['key','propensity_logit', 'base_year', 'target_year']]
    #   non_treated_df = non_treated_df.reset_index()
      if len(non_treated_df)>0:
        dict = {'propensity_logit': row['propensity_logit']}
        row_df = pd.DataFrame(dict, index = [row['key']])
        # print(row_df)
        # print('row df dtype',row_df.dtype)
        non_treated_df = non_treated_df[['key','propensity_logit']].set_index('key')
        # print(non_treated_df)
        matched_df = get_matching_pairs(row_df, non_treated_df)
        matching = pd.DataFrame({'key': row_df.index, 'matched_key': matched_df.index})
        all_matchings = pd.concat([all_matchings, matching], axis=0)
      else:
        blocks_no_cp.add(baseyear)
        # print('block: ',block)
        # print('len(treated_df): ',len(treated_df))
        # print('len(non_treated_df): ',len(non_treated_df))
  else:
    blocks_no_tp.add(baseyear)
    # print('block: ',block)
    # print('len(treated_df): ',len(treated_df))
    # print('len(non_treated_df): ',len(non_treated_df))

In [ ]:
matching = all_matchings
fields = ['key'] + metadata2 + confounding_vars
print(len(fields))

26


In [ ]:
matching

,key,matched_key
0,12847_2016_2020,25112_2016_2020
0,13666_2016_2020,26620_2016_2020
0,6536_2016_2021,46890_2016_2021
0,14545_2016_2019,34508_2016_2019
0,18663_2016_2018,96226_2016_2018
...,...,...
0,11624_2016_2021,56504_2016_2021
0,20309_2016_2019,69926_2016_2019
0,22172_2016_2019,45893_2016_2019
0,2843_2016_2021,64233_2016_2021


In [ ]:
df = matching.copy()

# Extract UIDs from 'key' column
df['UID'] = df['key'].str.split('_').str[0]

# Calculate unique UIDs
unique_uids = df['UID'].nunique()

# Print unique UIDs
print(f"Number of unique UIDs: {unique_uids}")

Number of unique UIDs: 2377


In [ ]:
matching.describe()

,key,matched_key
count,9508,9508
unique,9508,7327
top,12847_2016_2020,58910_2016_2020
freq,1,9


In [ ]:
all_fields = fields + [a+'_matched' for a in fields]
asset_cvts = pd.merge(matching, norm,  how='inner', left_on=['key'], right_on = ['key'])
asset_cfs_cvts= pd.merge(asset_cvts, norm,  how='inner', left_on=['matched_key'], right_on = ['key'])
asset_cfs_cvts
asset_cfs_cvts.to_csv("file_path", index=False)

In [ ]:
asset_cfs_cvts[f'did_ndvi_{season_name}'] = asset_cfs_cvts[f'{season_name}_ndvi_diff_x'] - asset_cfs_cvts[f'{season_name}_ndvi_diff_y']
asset_cfs_cvts[f'did_ndmi_{season_name}'] = asset_cfs_cvts[f'{season_name}_ndmi_diff_x'] - asset_cfs_cvts[f'{season_name}_ndmi_diff_y']
asset_cfs_cvts[f'did_GCI_{season_name}'] = asset_cfs_cvts[f'{season_name}_GCI_diff_x'] - asset_cfs_cvts[f'{season_name}_GCI_diff_y']

In [ ]:
asset_cfs_cvts[f'did_ndvi_{season_name}_norm'] = asset_cfs_cvts[f'{season_name}_ndvi_diff_norm_x'] - asset_cfs_cvts[f'{season_name}_ndvi_diff_norm_y']
asset_cfs_cvts[f'did_ndmi_{season_name}_norm'] = asset_cfs_cvts[f'{season_name}_ndmi_diff_norm_x'] - asset_cfs_cvts[f'{season_name}_ndmi_diff_norm_y']
asset_cfs_cvts[f'did_GCI_{season_name}_norm'] = asset_cfs_cvts[f'{season_name}_GCI_diff_norm_x'] - asset_cfs_cvts[f'{season_name}_GCI_diff_norm_y']

In [ ]:
results = pd.DataFrame(columns = ['Setting', 'Outcome', 'ATE', 'BaseYearOut_treated', 'TargetYearOut_treated', 'Difference_treated', 'BaseYearOut_control', 'TargetYearOut_control', 'Difference_control', 'p-value'])

In [ ]:
asset_cfs_cvts

,key_x,matched_key,UID_x,lat_x,lon_x,Treatment_x,Asset ID_x,objectid_x,dist_closest_crop_x,dist_closest_lin_x,dist_closest_river_x,dist_closest_upstream_forest_x,elevation_x,flow_accumulation_x,proximity_water_x,slope_x,HSG_x,pH_x,OC_x,CEC_x,recent_cropping_intensity_x,base_year_x,target_year_x,ndvi_Zaid_target_x,ndvi_Zaid_target_norm_x,ndvi_Zaid_base_x,ndvi_Zaid_base_norm_x,GCI_Zaid_target_x,GCI_Zaid_target_norm_x,GCI_Zaid_base_x,GCI_Zaid_base_norm_x,ndmi_Zaid_base_x,ndmi_Zaid_base_norm_x,ndmi_Zaid_target_x,ndmi_Zaid_target_norm_x,Zaid_ndvi_diff_x,Zaid_ndvi_diff_norm_x,Zaid_ndmi_diff_x,Zaid_ndmi_diff_norm_x,Zaid_GCI_diff_x,Zaid_GCI_diff_norm_x,precip_baseyr_Zaid_x,precip_targetyr_Zaid_x,precip_baseyr_Kharif_x,precip_targetyr_Kharif_x,precip_baseyr_Rabi_x,precip_targetyr_Rabi_x,HSG_3_x,HSG_4_x,HSG_13_x,HSG_14_x,UID_y,lat_y,lon_y,Treatment_y,Asset ID_y,objectid_y,dist_closest_crop_y,dist_closest_lin_y,dist_closest_river_y,dist_closest_upstream_forest_y,elevation_y,flow_accumulation_y,proximity_water_y,slope_y,HSG_y,pH_y,OC_y,CEC_y,recent_cropping_intensity_y,base_year_y,target_year_y,ndvi_Zaid_target_y,ndvi_Zaid_target_norm_y,ndvi_Zaid_base_y,ndvi_Zaid_base_norm_y,GCI_Zaid_target_y,GCI_Zaid_target_norm_y,GCI_Zaid_base_y,GCI_Zaid_base_norm_y,ndmi_Zaid_base_y,ndmi_Zaid_base_norm_y,ndmi_Zaid_target_y,ndmi_Zaid_target_norm_y,Zaid_ndvi_diff_y,Zaid_ndvi_diff_norm_y,Zaid_ndmi_diff_y,Zaid_ndmi_diff_norm_y,Zaid_GCI_diff_y,Zaid_GCI_diff_norm_y,precip_baseyr_Zaid_y,precip_targetyr_Zaid_y,precip_baseyr_Kharif_y,precip_targetyr_Kharif_y,precip_baseyr_Rabi_y,precip_targetyr_Rabi_y,HSG_3_y,HSG_4_y,HSG_13_y,HSG_14_y,key_y,did_ndvi_Zaid,did_ndmi_Zaid,did_GCI_Zaid,did_ndvi_Zaid_norm,did_ndmi_Zaid_norm,did_GCI_Zaid_norm
0,12847_2016_2020,25112_2016_2020,12847.0,24.352310,85.982447,1.0,3.400089e+10,2028.0,0.000000,0.304837,-0.324090,-0.357287,0.768561,-0.974124,0.505901,-0.527148,4.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.116945,0.562381,0.109419,0.413036,0.408873,0.340276,0.411786,0.358533,-0.031376,0.469570,0.025851,0.999907,0.007527,0.149345,0.057228,0.530337,-0.002913,-0.018256,-2.476260,0.137784,601.958832,877.048239,0.163029,-1.464819,0.0,1.0,0.0,0.0,25112.0,23.857682,86.465047,0.0,0.0,2101.0,178.252021,-2.268891,-1.632092,-1.742896,-0.107533,-0.309931,-0.460891,-0.280592,13.0,0.374073,1.086967,0.440463,1.0,2016.0,2020.0,0.116618,0.000000,0.123293,0.150853,0.328304,0.027096,0.344228,0.162720,-0.029863,0.049101,0.010814,0.621291,-0.006675,-0.150853,0.040677,0.572190,-0.015924,-0.135624,0.061542,0.567239,696.695966,770.142048,0.440132,-1.511248,0.0,0.0,1.0,0.0,25112_2016_2020,0.014202,0.016551,0.013011,0.300198,-0.041853,0.117368
1,13666_2016_2020,26620_2016_2020,13666.0,24.079788,86.292278,1.0,3.400097e+10,2054.0,60.295648,-1.444907,-0.533738,-0.935615,-0.003682,0.069238,1.159696,-0.250703,14.0,-0.181999,-0.723411,0.549438,1.0,2016.0,2020.0,0.164903,0.674914,0.133089,0.000000,0.493146,0.639235,0.404250,0.000000,-0.009286,0.286936,-0.014350,0.232087,0.031814,0.674914,-0.005064,-0.054849,0.088896,0.639235,-0.976481,0.318754,577.221264,865.177002,-0.146142,-1.541058,0.0,0.0,0.0,1.0,26620.0,23.904574,86.462352,0.0,0.0,2101.0,26.982975,-2.268891,-0.937659,0.848583,0.106258,0.069238,1.005818,-0.048994,3.0,1.737654,1.685346,-2.148870,1.0,2016.0,2020.0,0.155193,0.672125,0.100995,0.186001,0.469874,0.902330,0.323182,0.327995,-0.083141,0.000000,-0.061405,0.201805,0.054198,0.486124,0.021736,0.201805,0.146692,0.574335,0.061542,0.567239,696.695966,770.142048,0.440132,-1.511248,1.0,0.0,0.0,0.0,26620_2016_2020,-0.022383,-0.026800,-0.057796,0.188790,-0.256654,0.064900
2,6536_2016_2021,46890_2016_2021,6536.0,23.981077,86.476573,1.0,3.400120e+10,2064.0,0.000000,-1.027938,0.872949,-0.275503,-0.047996,-0.309931,0.001460,-0.207741,3.0,1.737654,1.685346,-2.148870,2.0,2016.0,2021.0,0.199449,0.999906,0.148504,0.521180,0.542436,0.999964,0.417131,0.544369,0.017200,0.485149,0.034164,0.671976,0.050946,0.478726,0.016964,0.186827,0.125306,0.455595,-0.336289,-1.29

In [ ]:
# code adding difference treated and difference control, normal yrs, drought yrs outcomes

metric = 'mean_of_mean'
results = pd.DataFrame()  # Initialize an empty DataFrame

outcomes = ['ndvi', 'ndmi', 'GCI']

for outcome in outcomes:
    dict_results = {}
    dict_results['Outcome'] = outcome
    dict_results['ATE'] = asset_cfs_cvts['did_' + outcome + f'_{season_name}'].mean()
    dict_results['TargetYearOut_treated'] = asset_cfs_cvts[outcome + f'_{season_name}_target_x'].mean()
    dict_results['BaseYearOut_treated'] = asset_cfs_cvts[outcome + f'_{season_name}_base_x'].mean()
    dict_results['Difference_treated'] = asset_cfs_cvts[f'{season_name}_' + outcome + '_diff_x'].mean()
    dict_results['TargetYearOut_control'] = asset_cfs_cvts[outcome + f'_{season_name}_target_y'].mean()
    dict_results['BaseYearOut_control'] = asset_cfs_cvts[outcome + f'_{season_name}_base_y'].mean()
    dict_results['Difference_control'] = asset_cfs_cvts[f'{season_name}_' + outcome + '_diff_y'].mean()
    dict_results['Farm Ponds/Observations'] = '2500/10000'

    result = ttest_ind(asset_cfs_cvts[f'{season_name}_' + outcome + '_diff_x'],
                       asset_cfs_cvts[f'{season_name}_' + outcome + '_diff_y'])

    dict_results['p-value'] = result.pvalue

    # Append dict_results to results DataFrame using pd.concat()
    results = pd.concat([results, pd.DataFrame([dict_results])], ignore_index=True)

results

,Outcome,ATE,TargetYearOut_treated,BaseYearOut_treated,Difference_treated,TargetYearOut_control,BaseYearOut_control,Difference_control,Farm Ponds/Observations,p-value
0,ndvi,-0.000750,0.166123,0.137038,0.029084,0.166129,0.136295,0.029834,2500/10000,0.121017
1,ndmi,0.000847,0.018967,-0.011211,0.030178,0.018769,-0.010561,0.029331,2500/10000,0.165714
2,GCI,-0.002610,0.466087,0.417174,0.048913,0.463784,0.412260,0.051524,2500/10000,0.039304


In [ ]:
results.to_csv(f"DiD_results_{season_name}_{str_date}.csv", index=False)